In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

### Код из библиотеки dlnlputils репозитория https://github.com/Samsung-IT-Academy/stepik-dl-nlp

In [ ]:
#########stepik-dl-nlp/dlnlputils/data/base.py#########

import collections
import re

import numpy as np

TOKEN_RE = re.compile(r'[\w\d]+')


def tokenize_text_simple_regex(txt, min_token_size=4):
    txt = txt.lower()
    all_tokens = TOKEN_RE.findall(txt)
    return [token for token in all_tokens if len(token) >= min_token_size]

def character_tokenize(txt):
    return list(txt)

def tokenize_corpus(texts, tokenizer=tokenize_text_simple_regex, **tokenizer_kwargs):
    return [tokenizer(text, **tokenizer_kwargs) for text in texts]

def tokenize_corpus_verbose(texts, tokenizer=tokenize_text_simple_regex, verbose_chunk=1000, **tokenizer_kwargs):
    tokenize_texts = []
    for i, text in enumerate(texts):
        tokenize_texts.append(tokenizer(text, **tokenizer_kwargs))
        if i % verbose_chunk == 0:
            print('Complete: {}/{}'.format(i,len(texts)))
    return tokenize_texts

def texts_to_token_ids(tokenized_texts, word2id):
    return [[word2id[token] for token in text if token in word2id]
            for text in tokenized_texts]


def build_vocabulary(tokenized_texts, max_size=1000000, max_doc_freq=0.8, min_count=5, pad_word=None, start_end_tag = False):
    #modified add start/end tags of words
    word_counts = collections.defaultdict(int)
    doc_n = 0

    # посчитать количество документов, в которых употребляется каждое слово
    # а также общее количество документов
    for txt in tokenized_texts:
        doc_n += 1
        unique_text_tokens = set(txt)
        for token in unique_text_tokens:
            word_counts[token] += 1

    # убрать слишком редкие и слишком частые слова
    word_counts = {word: cnt for word, cnt in word_counts.items()
                   if cnt >= min_count and cnt / doc_n <= max_doc_freq}

    # отсортировать слова по убыванию частоты
    sorted_word_counts = sorted(word_counts.items(),
                                reverse=True,
                                key=lambda pair: pair[1])
    
    # добавим теги начала и конца слова, для задачи POS-tagging
    if start_end_tag and pad_word is not None:
        sorted_word_counts = [('<START>', 1)] + [('<END>', 2)] + sorted_word_counts
    # добавим несуществующее слово с индексом 0 для удобства пакетной обработки
    if pad_word is not None:
        sorted_word_counts = [(pad_word, 0)] + sorted_word_counts

    # если у нас по прежнему слишком много слов, оставить только max_size самых частотных
    if len(word_counts) > max_size:
        sorted_word_counts = sorted_word_counts[:max_size]

    # нумеруем слова
    word2id = {word: i for i, (word, _) in enumerate(sorted_word_counts)}

    # нормируем частоты слов
    word2freq = np.array([cnt / doc_n for _, cnt in sorted_word_counts], dtype='float32')

    return word2id, word2freq



#########stepik-dl-nlp/dlnlputils/data/bag_of_words.py#########

import numpy as np
import scipy.sparse
import torch
from torch.utils.data import Dataset


def vectorize_texts(tokenized_texts, word2id, word2freq, mode='tfidf', scale=True):
    #modified by me 
    #add 'lftidf', 'tflidf', 'ltflidf', 'ltf', 'lidf'
    
    assert mode in {'tfidf', 'idf', 'tf', 'bin', 'ltfidf', 'tflidf', 'tflidf_v2', 'ltf', 'tfpmi'}

    # считаем количество употреблений каждого слова в каждом документе
    result = scipy.sparse.dok_matrix((len(tokenized_texts), len(word2id)), dtype='float32')
    for text_i, text in enumerate(tokenized_texts):
        for token in text:
            if token in word2id:
                result[text_i, word2id[token]] += 1

    # получаем бинарные вектора "встречается или нет"
    if mode == 'bin':
        result = (result > 0).astype('float32')

    # получаем вектора относительных частот слова в документе
    elif mode == 'tf':
        result = result.tocsr()
        result = result.multiply(1 / result.sum(1))

    # полностью убираем информацию о количестве употреблений слова в данном документе,
    # но оставляем информацию о частотности слова в корпусе в целом
    elif mode == 'idf':
        result = (result > 0).astype('float32').multiply(1 / word2freq)

    # учитываем всю информацию, которая у нас есть:
    # частоту слова в документе и частоту слова в корпусе
    elif mode == 'tfidf':
        result = result.tocsr()
        result = result.multiply(1 / result.sum(1))  # разделить каждую строку на её длину
        result = result.multiply(1 / word2freq)  # разделить каждый столбец на вес слова

    elif mode == 'ltf': # lTF=ln⁡(TF+1)
        result = result.tocsr()
        result = result.multiply(1 / result.sum(1))
        result = scipy.sparse.dok_matrix(np.log(result.toarray()+1))
 
    elif mode == 'lidf': # lIDF=ln⁡(n/IDF+1)
        result = (result > 0).astype('float32').multiply(len(tokenized_texts) / word2freq)
        result = scipy.sparse.dok_matrix(np.log(result.toarray()+1))

        
    elif mode == 'ltfidf': # lTFIDF=ln⁡(TF+1)⋅IDF
        result = result.tocsr() #переводим матрицу в режим быстрой работы со строками (это очень важно!!)
        result = result.multiply(1/result.sum(1)) # разделить каждую строку на её длину
        result = scipy.sparse.dok_matrix(np.log(result.toarray()+1))
        result = result.multiply(1 / word2freq) # разделить каждый столбец на вес слова
        

    elif mode == 'tflidf': # lTFIDF=TF⋅ln⁡(1/IDF+1)
        result = result.tocsr() #переводим матрицу в режим быстрой работы со строками (это очень важно!!)
        result = result.multiply(1/result.sum(1)) # разделить каждую строку на её длину
        result = result.multiply(np.log(1 / word2freq + 1)) # разделить каждый столбец на вес слова

    elif mode == 'tflidf_v2': # lTFIDF=TF⋅ln⁡(n/IDF+1)
        result = result.tocsr() #переводим матрицу в режим быстрой работы со строками (это очень важно!!)
        result = result.multiply(1/result.sum(1)) # разделить каждую строку на её длину
        result = result.multiply(np.log(len(tokenized_texts) / word2freq + 1)) # разделить каждый столбец на вес слова
        
    elif mode == 'tfpmi': # TFPMI=TF⋅PMI
        result = result.tocsr()
        result = result.multiply(1 / result.sum(1))  # разделить каждую строку на её длину
        result = result.multiply(word2freq)  # домножить каждую строку на word2freq (это массив PMI Scores)

    if scale:
        result = result.tocsc()
        result -= result.min()
        result /= (result.max() + 1e-6)

    return result.tocsr()


class SparseFeaturesDataset(Dataset):
    def __init__(self, features, targets):
        self.features = features
        self.targets = targets

    def __len__(self):
        return self.features.shape[0]

    def __getitem__(self, idx):
        cur_features = torch.from_numpy(self.features[idx].toarray()[0]).float()
        cur_label = torch.from_numpy(np.asarray(self.targets[idx])).long()
        return cur_features, cur_label
    
    
#########stepik-dl-nlp/dlnlputils/pipeline.py#########

import copy
import datetime
import random
import traceback

import numpy as np
import torch
from torch.utils.data import DataLoader


def init_random_seed(value=0):
    random.seed(value)
    np.random.seed(value)
    torch.manual_seed(value)
    torch.cuda.manual_seed(value)
    torch.backends.cudnn.deterministic = True


def copy_data_to_device(data, device):
    if torch.is_tensor(data):
        return data.to(device)
    elif isinstance(data, (list, tuple)):
        return [copy_data_to_device(elem, device) for elem in data]
    raise ValueError('Недопустимый тип данных {}'.format(type(data)))


def print_grad_stats(model):
    mean = 0
    std = 0
    norm = 1e-5
    for param in model.parameters():
        grad = getattr(param, 'grad', None)
        if grad is not None:
            mean += grad.data.abs().mean()
            std += grad.data.std()
            norm += 1
    mean /= norm
    std /= norm
    print(f'Mean grad {mean}, std {std}, n {norm}')


def train_eval_loop(model, train_dataset, val_dataset, criterion,
                    lr=1e-4, epoch_n=10, batch_size=32,
                    device=None, early_stopping_patience=10, l2_reg_alpha=0,
                    max_batches_per_epoch_train=10000,
                    max_batches_per_epoch_val=1000,
                    data_loader_ctor=DataLoader,
                    optimizer_ctor=None,
                    lr_scheduler_ctor=None,
                    shuffle_train=True,
                    dataloader_workers_n=0,
                    best_acc_type = 'loss',
                    test_dataset = None,
                    experiment_name = 'NoName',
                    no_calculate_accuracy = False):
    """
    v2.1
    Цикл для обучения модели. После каждой эпохи качество модели оценивается по отложенной выборке.
    :param model: torch.nn.Module - обучаемая модель
    :param train_dataset: torch.utils.data.Dataset - данные для обучения
    :param val_dataset: torch.utils.data.Dataset - данные для оценки качества
    :param criterion: функция потерь для настройки модели
    :param lr: скорость обучения
    :param epoch_n: максимальное количество эпох
    :param batch_size: количество примеров, обрабатываемых моделью за одну итерацию
    :param device: cuda/cpu - устройство, на котором выполнять вычисления
    :param early_stopping_patience: наибольшее количество эпох, в течение которых допускается
        отсутствие улучшения модели, чтобы обучение продолжалось.
    :param l2_reg_alpha: коэффициент L2-регуляризации
    :param max_batches_per_epoch_train: максимальное количество итераций на одну эпоху обучения
    :param max_batches_per_epoch_val: максимальное количество итераций на одну эпоху валидации
    :param data_loader_ctor: функция для создания объекта, преобразующего датасет в батчи
        (по умолчанию torch.utils.data.DataLoader)
    :return: кортеж из двух элементов:
        - среднее значение функции потерь на валидации на лучшей эпохе
        - лучшая модель
    """
    
    '''
    modified by wisoffe
    best_acc_type: 'loss' or 'acc'
    experiment_name: 
    '''
    assert best_acc_type in {'loss', 'acc'}
    
    train_start_time = datetime.datetime.now()
    print("############## Start experiment with name: {} ##############".format(experiment_name))
    
    #statistics history
    history = {'acc': {'train': [0.0],
                       'val': [0.0]},
               'loss': {'train': [float('inf')],
                       'val': [float('inf')]}}
    
    if device is None:
        device = 'cuda' if torch.cuda.is_available() else 'cpu'
    device = torch.device(device)
    model.to(device)

    if optimizer_ctor is None:
        optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=l2_reg_alpha)
    else:
        optimizer = optimizer_ctor(model.parameters(), lr=lr)

    if lr_scheduler_ctor is not None:
        lr_scheduler = lr_scheduler_ctor(optimizer)
    else:
        lr_scheduler = None

    train_dataloader = data_loader_ctor(train_dataset, batch_size=batch_size, shuffle=shuffle_train,
                                        num_workers=dataloader_workers_n)
    val_dataloader = data_loader_ctor(val_dataset, batch_size=batch_size, shuffle=False,
                                      num_workers=dataloader_workers_n)
    
    if best_acc_type == 'loss': #отбираем модель по минимальному loss
        best_val_metric = float('inf')
    elif best_acc_type == 'acc': #отбираем модель по максимальному accuracy
        best_val_metric = float('-inf')
        
    best_epoch_i = 0
    best_model = copy.deepcopy(model)
    
    
    for epoch_i in range(1, epoch_n + 1):
        try:
            #####train phase######
            epoch_start = datetime.datetime.now()
            train_accuracy_epoch = [] #for statistics
            train_loss_epoch = [] #for statistics
            
            model.train()
            
            for batch_i, (batch_x, batch_y) in enumerate(train_dataloader):
                if batch_i > max_batches_per_epoch_train:
                    print('Threshold max_batches_per_epoch_train exceeded!')
                    break

                batch_x = copy_data_to_device(batch_x, device)
                batch_y = copy_data_to_device(batch_y, device)

                pred = model(batch_x)
                loss = criterion(pred, batch_y)

                model.zero_grad()
                loss.backward()

                optimizer.step()

                train_loss_epoch.append(float(loss))
                
                if not no_calculate_accuracy:
                    train_accuracy_epoch.append(float((pred.argmax(dim=1) == batch_y.data).float().mean().data))
                    #train_accuracy_epoch.append(float((pred.detach().cpu().numpy().argmax(-1) == batch_y.detach().cpu().numpy()).mean()))
                else: train_accuracy_epoch.append(0.)
                    
            
            #####validation phase######
            model.eval()

            val_accuracy_epoch = [] #for statistics
            val_loss_epoch = [] #for statistics

            with torch.no_grad():
                for batch_i, (batch_x, batch_y) in enumerate(val_dataloader):
                    if batch_i > max_batches_per_epoch_val:
                        print('Threshold max_batches_per_epoch_val exceeded!')
                        break

                    batch_x = copy_data_to_device(batch_x, device)
                    batch_y = copy_data_to_device(batch_y, device)

                    pred = model(batch_x)
                    loss = criterion(pred, batch_y)
                    
                    if not no_calculate_accuracy:
                        val_accuracy_epoch.append(float((pred.argmax(dim=1) == batch_y.data).float().mean().data))
                        #val_accuracy_epoch.append(float((pred.detach().cpu().numpy().argmax(-1) == batch_y.detach().cpu().numpy()).mean()))
                    else:
                        val_accuracy_epoch.append(0.)
                    val_loss_epoch.append(float(loss))

            
            ########ending of epoch#########
            
            history['acc']['train'].append(sum(train_accuracy_epoch) / len(train_accuracy_epoch))
            history['loss']['train'].append(sum(train_loss_epoch) / len(train_loss_epoch))  

            history['acc']['val'].append(sum(val_accuracy_epoch) / len(val_accuracy_epoch))
            history['loss']['val'].append(sum(val_loss_epoch) / len(val_loss_epoch))
            
            
            #save best model
            best_model_saved = False
            if (best_acc_type == 'loss' and history['loss']['val'][-1] < best_val_metric) or \
                    (best_acc_type == 'acc' and history['acc']['val'][-1] > best_val_metric):
                #отбираем модель по минимальному loss или максимальному accuracy
                best_epoch_i = epoch_i
                best_val_metric = history[best_acc_type]['val'][-1]
                best_model = copy.deepcopy(model)
                best_model_saved = True
            #check for break training
            elif epoch_i - best_epoch_i > early_stopping_patience:
                print('Модель не улучшилась за последние {} эпох, прекращаем обучение'.format(
                    early_stopping_patience))
                break

            if lr_scheduler is not None:
                lr_scheduler.step(history['loss']['val'][-1])
            
            #output statistics
            
            print('Epoch = {:>3},   ACC: val = {:.4f}, train = {:.4f}    LOSS: val = {:.4f}, train = {:.4f}   SAVE: {}, Time: {:0.2f}s'\
                  .format(epoch_i,
                          history['acc']['val'][-1], 
                          history['acc']['train'][-1],
                          history['loss']['val'][-1],
                          history['loss']['train'][-1],
                          best_model_saved,
                          (datetime.datetime.now() - epoch_start).total_seconds()),
                  flush=True)

        except KeyboardInterrupt:
            print('Досрочно остановлено пользователем')
            break
        except Exception as ex:
            print('Ошибка при обучении: {}\n{}'.format(ex, traceback.format_exc()))
            break
            
    print(' ')
    print("BEST MODEL: ACC: val = {:.4f}, train = {:.4f}, LOSS: val = {:.4f}, train = {:.4f}, on epoch = {}, metric type = {}, Full train time = {:0.2f}s"\
                  .format(history['acc']['val'][best_epoch_i], 
                          history['acc']['train'][best_epoch_i],
                          history['loss']['val'][best_epoch_i],
                          history['loss']['train'][best_epoch_i],
                          best_epoch_i,
                          best_acc_type,
                          (datetime.datetime.now() - train_start_time).total_seconds()))
    print("************** End experiment with name: {} **************".format(experiment_name))
    print(' ')
    history['BEST'] = {}
    history['BEST']['epoch'] = best_epoch_i
    history['BEST']['dict_size'] = batch_x.shape[-1]
    
    
    #calculate and save final metrics best_model on train/val/test datasets
    if test_dataset is not None:
        history['BEST']['acc'] = {}
        history['BEST']['loss'] = {}
        
        #save validation metrics (no calculate again)
        history['BEST']['acc']['val'] = history['acc']['val'][best_epoch_i]
        history['BEST']['loss']['val'] = history['loss']['val'][best_epoch_i]
        
        #calculate and save train metrics
        train_pred = predict_with_model(best_model, train_dataset, return_labels=True)
        history['BEST']['loss']['train'] = float(F.cross_entropy(torch.from_numpy(train_pred[0]),
                             torch.from_numpy(train_pred[1]).long()))
        history['BEST']['acc']['train'] = accuracy_score(train_pred[1], train_pred[0].argmax(-1))
        
        #calculate and save test metrics
        test_pred = predict_with_model(best_model, test_dataset, return_labels=True)
        history['BEST']['loss']['test'] = float(F.cross_entropy(torch.from_numpy(test_pred[0]),
                             torch.from_numpy(test_pred[1]).long()))
        history['BEST']['acc']['test'] = accuracy_score(test_pred[1], test_pred[0].argmax(-1))    
    
    
    return history, best_model


def predict_with_model(model, dataset, device=None, batch_size=32, num_workers=0, return_labels=False):
    """
    :param model: torch.nn.Module - обученная модель
    :param dataset: torch.utils.data.Dataset - данные для применения модели
    :param device: cuda/cpu - устройство, на котором выполнять вычисления
    :param batch_size: количество примеров, обрабатываемых моделью за одну итерацию
    :return: numpy.array размерности len(dataset) x *
    """
    if device is None:
        device = 'cuda' if torch.cuda.is_available() else 'cpu'
    results_by_batch = []

    device = torch.device(device)
    model.to(device)
    model.eval()

    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers)
    labels = []
    with torch.no_grad():
        import tqdm
        for batch_x, batch_y in tqdm.tqdm(dataloader, total=len(dataset)/batch_size):
            batch_x = copy_data_to_device(batch_x, device)

            if return_labels:
                labels.append(batch_y.numpy())

            batch_pred = model(batch_x)
            results_by_batch.append(batch_pred.detach().cpu().numpy())

    if return_labels:
        return np.concatenate(results_by_batch, 0), np.concatenate(labels, 0)
    else:
        return np.concatenate(results_by_batch, 0)


#########stepik-dl-nlp/dlnlputils/nnets.py#########

from torch.utils.data import Dataset


def ensure_length(txt, out_len, pad_value):
    if len(txt) < out_len:
        txt = list(txt) + [pad_value] * (out_len - len(txt))
    else:
        txt = txt[:out_len]
    return txt


class PaddedSequenceDataset(Dataset):
    def __init__(self, texts, targets, out_len=100, pad_value=0):
        self.texts = texts
        self.targets = targets
        self.out_len = out_len
        self.pad_value = pad_value

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, item):
        txt = self.texts[item]

        txt = ensure_length(txt, self.out_len, self.pad_value)
        txt = torch.tensor(txt, dtype=torch.long)

        target = torch.tensor(self.targets[item], dtype=torch.long)

        return txt, target

#########stepik-dl-nlp/dlnlputils/embeddings.py#########

class Embeddings:
    def __init__(self, embeddings, word2id):
        self.embeddings = embeddings
        self.embeddings /= (np.linalg.norm(self.embeddings, ord=2, axis=-1, keepdims=True) + 1e-4)
        self.word2id = word2id
        self.id2word = {i: w for w, i in word2id.items()}

    def most_similar(self, positive=None, negative=None, topk=10, with_mean = False):
        #modified by wis, converted to gensim syntax
        
        if positive is not None:
            if type(positive) != list:
                positive = [positive]
            pos_vec = [self.get_vector(word) for word in positive]
            pos_len = len(positive)
        else:
            pos_vec = 0
            pos_len = 1
            
        if negative is not None:
            if type(negative) != list:
                negative = [negative]
            neg_vec = [self.get_vector(word) for word in negative]
            neg_len = len(negative)
        else:
            neg_vec = 0
            neg_len = 1
        
        if with_mean:
            result_vec = np.array(pos_vec).sum(0) / pos_len - np.array(neg_vec).sum(0) / neg_len
        else:
            result_vec = np.array(pos_vec).sum(0) - np.array(neg_vec).sum(0)
        
        return self.most_similar_by_vector(result_vec, topk=topk)
    
    def most_similar_legacy(self, word, topk=10):
        return self.most_similar_by_vector(self.get_vector(word), topk=topk)

    def analogy(self, a1, b1, a2, topk=10):
        a1_v = self.get_vector(a1)
        b1_v = self.get_vector(b1)
        a2_v = self.get_vector(a2)
        query = b1_v - a1_v + a2_v
        return self.most_similar_by_vector(query, topk=topk)

    def most_similar_by_vector(self, query_vector, topk=10):
        similarities = (self.embeddings * query_vector).sum(-1)
        best_indices = np.argpartition(-similarities, topk, axis=0)[:topk]
        result = [(self.id2word[i], similarities[i]) for i in best_indices]
        result.sort(key=lambda pair: -pair[1])
        return result

    def get_vector(self, word):
        if word not in self.word2id:
            raise ValueError('Неизвестное слово "{}"'.format(word))
        return self.embeddings[self.word2id[word]]

    def get_vectors(self, *words):
        word_ids = [self.word2id[i] for i in words]
        vectors = np.stack([self.embeddings[i] for i in word_ids], axis=0)
        return vectors

#########stepik-dl-nlp/dlnlputils/visualization.py#########

from sklearn.decomposition import TruncatedSVD
from sklearn.manifold import TSNE


def plot_vectors(vectors, labels, how='tsne', ax=None, xy_lim=None):
    if how == 'tsne':
        projections = TSNE().fit_transform(vectors)
    elif how == 'svd':
        projections = TruncatedSVD().fit_transform(vectors)

    x = projections[:, 0]
    y = projections[:, 1]
    if xy_lim is not None:
        ax.set_xlim(xy_lim)
        ax.set_ylim(xy_lim)
    ax.scatter(x, y)
    for cur_x, cur_y, cur_label in zip(x, y, labels):
        ax.annotate(cur_label, (cur_x, cur_y))
        

#########stepik-dl-nlp/dlnlputils/data/pos.py#########
import torch
from torch.utils.data import TensorDataset

def pos_corpus_to_tensor(sentences, char2id, label2id, max_sent_len, max_token_len):
    #v1.2 add custom start/end tag functionaly
    inputs = torch.zeros((len(sentences), max_sent_len, max_token_len + 2), dtype=torch.long)
    inputs[:,:,0] = char2id.get('<START>', 0)
    targets = torch.zeros((len(sentences), max_sent_len), dtype=torch.long)

    for sent_i, sent in enumerate(sentences):
        for token_i, token in enumerate(sent):
            targets[sent_i, token_i] = label2id.get(token.upos, 0)
            if token.form is not None:
                for char_i, char in enumerate(token.form):
                    inputs[sent_i, token_i, char_i + 1] = char2id.get(char, 0)
            else:
                for char_i, char in enumerate('-'):
                    inputs[sent_i, token_i, char_i + 1] = char2id.get(char, 0)
            inputs[sent_i, token_i, char_i + 2] = char2id.get('<END>', 0)

    return inputs, targets


class POSTagger:
    def __init__(self, model, char2id, id2label, max_sent_len, max_token_len):
        self.model = model
        self.char2id = char2id
        self.id2label = id2label
        self.max_sent_len = max_sent_len
        self.max_token_len = max_token_len

    def __call__(self, sentences):
        tokenized_corpus = tokenize_corpus(sentences, min_token_size=1)

        inputs = torch.zeros((len(sentences), self.max_sent_len, self.max_token_len + 2), dtype=torch.long)
        inputs[:,:,0] = self.char2id.get('<START>', 0)

        for sent_i, sentence in enumerate(tokenized_corpus):
            for token_i, token in enumerate(sentence):
                for char_i, char in enumerate(token):
                    inputs[sent_i, token_i, char_i + 1] = self.char2id.get(char, 0)
                inputs[sent_i, token_i, char_i + 2] = self.char2id.get('<END>', 0)

        dataset = TensorDataset(inputs, torch.zeros(len(sentences)))
        predicted_probs = predict_with_model(self.model, dataset)  # SentenceN x TagsN x MaxSentLen
        predicted_classes = predicted_probs.argmax(1)

        result = []
        for sent_i, sent in enumerate(tokenized_corpus):
            result.append([self.id2label[cls] for cls in predicted_classes[sent_i, :len(sent)]])
        return result

### Мои наработки

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
# import spacy
# !python -m spacy download ru_core_news_md
# spacy_nlp = spacy.load('ru_core_news_md', disable=['parser', 'ner'])

def tokenize_text_spacy_lemmatize(txt, spacy_nlp, min_token_size=4, with_pos = True, remove_stopwords = False):
    doc = spacy_nlp(txt)
    
    if remove_stopwords:
        lemmatized_doc = [token for token in doc if (len(token) >= min_token_size) and (not token.is_stop)]
    else:
        lemmatized_doc = [token for token in doc if len(token) >= min_token_size]
    
    if with_pos:
        return ['_'.join([token.lemma_, token.pos_]) for token in lemmatized_doc]
    else:
        return [token.lemma_ for token in lemmatized_doc]

def tokenize_corpus_convert(tokenized_corpus, converter, addition = False):
    '''
    Convert each token in tokenized_corpus by converter
    
    Sample (PorterStemmer):
    import nltk
    ps = nltk.stemmer.PorterStemmer()
    tokenized_stemmed_corpus = tokenize_corpus_convert(tokenized_corpus, converter=ps.stem)
    
    Sample (SnowballStemmer):
    import nltk
    sno = nltk.stem.SnowballStemmer('english')
    tokenized_stemmed_corpus = tokenize_corpus_convert(tokenized_corpus, converter=sno.stem)
    
    Sample (WordNetLemmatizer):
    import nltk
    lemma = nltk.wordnet.WordNetLemmatizer()
    tokenized_lemmas_corpus = tokenize_corpus_convert(tokenized_corpus, converter=lemma.lemmatize)
    '''
    output = []
    if not addition: #возвращаем только преобразованные токены
        for doc in tokenized_corpus:
            output.append([converter(token) for token in doc])
    else: #возвращаем списк из исходных токенов, дополненных списком преобразованных
        for doc in tokenized_corpus:
            output.append(doc + [converter(token) for token in doc])        
    return output

def show_experiments_stats(histories, figsize = (16.0, 6.0), show_plots = True, only_BEST_MODEL_CALC = False):
    matplotlib.rcParams['figure.figsize'] = figsize
    
    for experiment_id in histories.keys():
        print('{:-<100}'.format(experiment_id))
        
        if not only_BEST_MODEL_CALC:
            epoch_max_acc = np.array(histories[experiment_id]['acc']['val']).argmax()
            print('Max val acc on:    Epoch = {:>3},   ACCURACY: val  = {:.4f}, train = {:.4f},   LOSS: val  = {:.4f}, train = {:.4f}'\
                  .format(epoch_max_acc, 
                          histories[experiment_id]['acc']['val'][epoch_max_acc], 
                          histories[experiment_id]['acc']['train'][epoch_max_acc],
                          histories[experiment_id]['loss']['val'][epoch_max_acc],
                          histories[experiment_id]['loss']['train'][epoch_max_acc]))
            epoch_min_loss = np.array(histories[experiment_id]['loss']['val']).argmin()
            print('Min val loss on:   Epoch = {:>3},   ACCURACY: val  = {:.4f}, train = {:.4f},   LOSS: val  = {:.4f}, train = {:.4f}'\
                  .format(epoch_min_loss, 
                          histories[experiment_id]['acc']['val'][epoch_min_loss], 
                          histories[experiment_id]['acc']['train'][epoch_min_loss],
                          histories[experiment_id]['loss']['val'][epoch_min_loss],
                          histories[experiment_id]['loss']['train'][epoch_min_loss]))
        
        if 'acc' in histories[experiment_id]['BEST']:
            print("BEST MODEL CALC:   Epoch = {:>3},   ACCURACY: test = {:.4f}, train = {:.4f},   LOSS: test = {:.4f}, train = {:.4f}  DICT SIZE = {}"\
                  .format(histories[experiment_id]['BEST']['epoch'], 
                          histories[experiment_id]['BEST']['acc']['test'],
                          histories[experiment_id]['BEST']['acc']['train'],
                          histories[experiment_id]['BEST']['loss']['test'],
                          histories[experiment_id]['BEST']['loss']['train'],
                          histories[experiment_id]['BEST']['dict_size']))
    
    
    if show_plots:
        for experiment_id in histories.keys():
            plt.plot(histories[experiment_id]['acc']['val'], label=experiment_id + ' val')
        plt.legend()
        plt.title('Validation Accuracy (Val only)')
        plt.show()

        for experiment_id in histories.keys():
            plt.plot(histories[experiment_id]['acc']['val'], label=experiment_id + ' val')
            plt.plot(histories[experiment_id]['acc']['train'], label=experiment_id + ' train')
        plt.legend()
        plt.title('Validation Accuracy (Val/Train)');
        plt.show()

        for experiment_id in histories.keys():
            plt.plot(histories[experiment_id]['loss']['val'], label=experiment_id  + ' val')
        plt.legend()
        plt.title('Validation Loss (Val only)');
        plt.show()

        for experiment_id in histories.keys():
            plt.plot(histories[experiment_id]['loss']['val'], label=experiment_id  + ' val')
            plt.plot(histories[experiment_id]['loss']['train'], label=experiment_id  + ' train')
        plt.legend()
        plt.title('Validation Loss (Val/Train)');
        plt.show()

def run_most_sumilars(func_most_similars, words_list, verbose = True, **kwargs):
    most_similars = {word: func_most_similars(word, **kwargs) for word in words_list}
    if verbose:
        for word, similars in most_similars.items():
            print('{}:'.format(word))
            print('\n'.join(map(str,similars)))
            print(' ')
    return most_similars
        

#https://stackoverflow.com/questions/4529815/saving-an-object-data-persistence/4529901
import pickle
def save_object(obj, filename):
    with open(filename, 'wb') as outp:  # Overwrites any existing file.
        pickle.dump(obj, outp, pickle.HIGHEST_PROTOCOL)

def load_object(filename):
    with open(filename, 'rb') as inp:
        return pickle.load(inp)

# sample usage
#company1 = [1,2,3,4,5]
#save_object(company1, '/kaggle/working/company1.pkl')
#del company
#company1 = load_object(filename)

In [ ]:
histories = {} #history of expeiments
hdatasets = {} #history of datasets

# Свёрточные нейросети и POS-теггинг

POS-теггинг - определение частей речи (снятие частеречной неоднозначности)

In [ ]:
!pip install pyconll
# !pip install spacy_udpipe

In [ ]:
%load_ext autoreload
%autoreload 2

import warnings
warnings.filterwarnings('ignore')

import sys; sys.path.append('./stepik-dl-nlp')

from sklearn.metrics import classification_report, f1_score

import numpy as np

import pyconll

import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import TensorDataset

# import dlnlputils
# from dlnlputils.data import tokenize_corpus, build_vocabulary, \
#     character_tokenize, pos_corpus_to_tensor, POSTagger
# from dlnlputils.pipeline import train_eval_loop, predict_with_model, init_random_seed

init_random_seed()

## Загрузка текстов и разбиение на обучающую и тестовую подвыборки

In [ ]:
# Если Вы запускаете ноутбук на colab или kaggle, добавьте в начало пути ./stepik-dl-nlp
!mkdir ./stepik-dl-nlp/
!mkdir ./stepik-dl-nlp/datasets/
!wget -O ./stepik-dl-nlp/datasets/ru_syntagrus-ud-train.conllu https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-train-a.conllu
!wget -O ./stepik-dl-nlp/datasets/ru_syntagrus-ud-dev.conllu https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-dev.conllu

In [ ]:
# Если Вы запускаете ноутбук на colab или kaggle, добавьте в начало пути ./stepik-dl-nlp
full_train = pyconll.load_from_file('./stepik-dl-nlp/datasets/ru_syntagrus-ud-train.conllu')
full_test = pyconll.load_from_file('./stepik-dl-nlp/datasets/ru_syntagrus-ud-dev.conllu')
print(len(full_train), len(full_test))

In [ ]:
for sent in full_train[:2]:
    for token in sent:
        print(token.form, token.upos)
    print()

In [ ]:
MAX_SENT_LEN = max(len(sent) for sent in full_train)
MAX_ORIG_TOKEN_LEN = max(len(token.form) for sent in full_train for token in sent)
print('Наибольшая длина предложения', MAX_SENT_LEN)
print('Наибольшая длина токена', MAX_ORIG_TOKEN_LEN)

In [ ]:
all_train_texts = [' '.join(token.form for token in sent) for sent in full_train]
print('\n'.join(all_train_texts[:10]))

In [ ]:
train_char_tokenized = tokenize_corpus(all_train_texts, tokenizer=character_tokenize)
char_vocab, word_doc_freq = build_vocabulary(train_char_tokenized, max_doc_freq=1.0, min_count=5, pad_word='<PAD>')
print("Количество уникальных символов", len(char_vocab))
print(list(char_vocab.items())[:10])

In [ ]:
UNIQUE_TAGS = ['<NOTAG>'] + sorted({token.upos for sent in full_train for token in sent if token.upos})
label2id = {label: i for i, label in enumerate(UNIQUE_TAGS)}
label2id

In [ ]:
train_inputs, train_labels = pos_corpus_to_tensor(full_train, char_vocab, label2id, MAX_SENT_LEN, MAX_ORIG_TOKEN_LEN)
train_dataset = TensorDataset(train_inputs, train_labels)

test_inputs, test_labels = pos_corpus_to_tensor(full_test, char_vocab, label2id, MAX_SENT_LEN, MAX_ORIG_TOKEN_LEN)
test_dataset = TensorDataset(test_inputs, test_labels)

In [ ]:
#SentCount x MAX_SENT_LEN x MAX_ORIG_TOKEN_LEN
train_inputs.shape

In [ ]:
train_inputs[1][:5]

In [ ]:
#SentCount x MAX_SENT_LEN
train_labels.shape

In [ ]:
train_labels[1]

In [ ]:
#for calc final statistics
hdatasets['base'] = {}
hdatasets['base']['char_vocab'] = char_vocab
hdatasets['base']['train_labels'] = train_labels
hdatasets['base']['train_dataset'] = train_dataset
hdatasets['base']['test_labels'] = test_labels
hdatasets['base']['test_dataset'] = test_dataset

## Вспомогательная свёрточная архитектура

In [ ]:
# FeaturesNum - изначально размер эмбеддингов (символов, либо слов), далее размерность сохраняется, 
# по сути это становится количеством входных каналов

class StackedConv1d(nn.Module):
    def __init__(self, features_num, layers_n=1, kernel_size=3, conv_layer=nn.Conv1d, dropout=0.0):
        super().__init__()
        layers = []
        for _ in range(layers_n):
            layers.append(nn.Sequential(
                conv_layer(features_num, features_num, kernel_size, padding=kernel_size//2),
                nn.Dropout(dropout),
                nn.LeakyReLU()))
        self.layers = nn.ModuleList(layers)
    
    def forward(self, x):
        """x - BatchSize x FeaturesNum x SequenceLen"""
        for layer in self.layers:
            x = x + layer(x)
        return x

## Предсказание частей речи на уровне отдельных токенов

In [ ]:
class SingleTokenPOSTagger(nn.Module):
    def __init__(self, vocab_size, labels_num, embedding_size=32, **kwargs):
        super().__init__()
        self.char_embeddings = nn.Embedding(vocab_size, embedding_size, padding_idx=0)
        self.backbone = StackedConv1d(embedding_size, **kwargs)
        self.global_pooling = nn.AdaptiveMaxPool1d(1)
        self.out = nn.Linear(embedding_size, labels_num)
        self.labels_num = labels_num
    
    def forward(self, tokens):
        """tokens - BatchSize x MaxSentenceLen x MaxTokenLen"""
        batch_size, max_sent_len, max_token_len = tokens.shape
        tokens_flat = tokens.view(batch_size * max_sent_len, max_token_len)
        
        char_embeddings = self.char_embeddings(tokens_flat)  # BatchSize*MaxSentenceLen x MaxTokenLen x EmbSize
        char_embeddings = char_embeddings.permute(0, 2, 1)  # BatchSize*MaxSentenceLen x EmbSize x MaxTokenLen
        
        features = self.backbone(char_embeddings)
        
        global_features = self.global_pooling(features).squeeze(-1)  # BatchSize*MaxSentenceLen x EmbSize
        
        logits_flat = self.out(global_features)  # BatchSize*MaxSentenceLen x LabelsNum
        logits = logits_flat.view(batch_size, max_sent_len, self.labels_num)  # BatchSize x MaxSentenceLen x LabelsNum
        logits = logits.permute(0, 2, 1)  # BatchSize x LabelsNum x MaxSentenceLen
        return logits

In [ ]:
# #hyperparameters
# exp_name = 'baseline_no_context'
# hyps = {
#     'dataset': 'base',
#     'char_min_count': 5,
#     'char_max_doc_freq': 1.0, 
#     'MAX_SENT_LEN': MAX_SENT_LEN,
#     'MAX_ORIG_TOKEN_LEN': MAX_ORIG_TOKEN_LEN,
    
#     #TokenPOSTagger
#     'emb_size': 64,
#     'sng_layers_n': 3, 
#     'sng_kernel_size': 3, 
#     'sng_dropout': 0.3,
#     'sng_conv_layer': nn.Conv1d,

#     #train_eval_loop
#     'lr': 5e-3,
#     'epoch_n': 30,
#     'batch': 64,
#     'device': 'cuda',
#     'stop_pat': 5,
#     'lr_sched_pat': 2,
#     'lr_sched_fac': 0.5
# }

# histories[exp_name] = {
#     'hyps': hyps
# }

# single_token_model = SingleTokenPOSTagger(len(char_vocab), len(label2id), embedding_size=hyps['emb_size'], layers_n=hyps['sng_layers_n'], 
#                                           kernel_size=hyps['sng_kernel_size'], dropout=hyps['sng_dropout'])
# print('Количество параметров', sum(np.product(t.shape) for t in single_token_model.parameters()))

# (histories[exp_name]['train_history'],
#  histories[exp_name]['best_model']) = train_eval_loop(single_token_model,
#                                             train_dataset,
#                                             test_dataset,
#                                             F.cross_entropy,
#                                             lr=hyps['lr'],
#                                             epoch_n=hyps['epoch_n'],
#                                             batch_size=hyps['batch'],
#                                             device=hyps['device'],
#                                             early_stopping_patience=hyps['stop_pat'],
#                                             max_batches_per_epoch_train=500,
#                                             max_batches_per_epoch_val=300,
#                                             experiment_name = exp_name,
#                                             lr_scheduler_ctor=lambda optim: torch.optim.lr_scheduler.ReduceLROnPlateau(optim, patience=hyps['lr_sched_pat'],
#                                                                                                                        factor=hyps['lr_sched_fac'],
#                                                                                                                        verbose=True))

In [ ]:
# # Если Вы запускаете ноутбук на colab или kaggle, добавьте в начало пути ./stepik-dl-nlp
# !mkdir ./stepik-dl-nlp/models/
# torch.save(histories[exp_name]['best_model'].state_dict(), './stepik-dl-nlp/models/single_token_pos.pth')

In [ ]:
# # Если Вы запускаете ноутбук на colab или kaggle, добавьте в начало пути ./stepik-dl-nlp
# histories[exp_name]['best_model'].load_state_dict(torch.load('./stepik-dl-nlp/models/single_token_pos.pth'))

## Предсказание частей речи на уровне предложений (с учётом контекста)

In [ ]:
class SentenceLevelPOSTagger(nn.Module):
    def __init__(self, vocab_size, labels_num, embedding_size=32, single_backbone_kwargs={}, context_backbone_kwargs={}):
        super().__init__()
        self.embedding_size = embedding_size
        self.char_embeddings = nn.Embedding(vocab_size, embedding_size, padding_idx=0)
        self.single_token_backbone = StackedConv1d(embedding_size, **single_backbone_kwargs)
        self.context_backbone = StackedConv1d(embedding_size, **context_backbone_kwargs)
        self.global_pooling = nn.AdaptiveMaxPool1d(1)
        self.out = nn.Conv1d(embedding_size, labels_num, 1)
        self.labels_num = labels_num
    
    def forward(self, tokens):
        """tokens - BatchSize x MaxSentenceLen x MaxTokenLen"""
        batch_size, max_sent_len, max_token_len = tokens.shape
        tokens_flat = tokens.view(batch_size * max_sent_len, max_token_len)
        
        char_embeddings = self.char_embeddings(tokens_flat)  # BatchSize*MaxSentenceLen x MaxTokenLen x EmbSize
        char_embeddings = char_embeddings.permute(0, 2, 1)  # BatchSize*MaxSentenceLen x EmbSize x MaxTokenLen
        char_features = self.single_token_backbone(char_embeddings)
        
        token_features_flat = self.global_pooling(char_features).squeeze(-1)  # BatchSize*MaxSentenceLen x EmbSize

        token_features = token_features_flat.view(batch_size, max_sent_len, self.embedding_size)  # BatchSize x MaxSentenceLen x EmbSize
        token_features = token_features.permute(0, 2, 1)  # BatchSize x EmbSize x MaxSentenceLen
        context_features = self.context_backbone(token_features)  # BatchSize x EmbSize x MaxSentenceLen

        logits = self.out(context_features)  # BatchSize x LabelsNum x MaxSentenceLen
        return logits

In [ ]:
# #hyperparameters
# exp_name = 'baseline_context'
# hyps = {
#     'dataset': 'base',
#     'char_min_count': 5,
#     'char_max_doc_freq': 1.0, 
#     'MAX_SENT_LEN': MAX_SENT_LEN,
#     'MAX_ORIG_TOKEN_LEN': MAX_ORIG_TOKEN_LEN,
    
#     #TokenPOSTagger
#     'emb_size': 64,
#     'sng_layers_n': 3, 
#     'sng_kernel_size': 3, 
#     'sng_dropout': 0.3,
#     'sng_conv_layer': nn.Conv1d,
#     'ctx_layers_n': 3, 
#     'ctx_kernel_size': 3, 
#     'ctx_dropout': 0.3,
#     'ctx_conv_layer': nn.Conv1d,

#     #train_eval_loop
#     'lr': 5e-3,
#     'epoch_n': 30,
#     'batch': 64,
#     'device': 'cuda',
#     'stop_pat': 5,
#     'lr_sched_pat': 2,
#     'lr_sched_fac': 0.5
# }

# histories[exp_name] = {
#     'hyps': hyps
# }

# sentence_level_model = SentenceLevelPOSTagger(len(char_vocab), len(label2id), embedding_size=hyps['emb_size'],
#                                               single_backbone_kwargs=dict(layers_n=hyps['sng_layers_n'], kernel_size=hyps['sng_kernel_size'], 
#                                                                           dropout=hyps['sng_dropout'], conv_layer=hyps['sng_conv_layer']),
#                                               context_backbone_kwargs=dict(layers_n=hyps['ctx_layers_n'], kernel_size=hyps['ctx_kernel_size'], 
#                                                                            dropout=hyps['ctx_dropout'], conv_layer=hyps['ctx_conv_layer']))
# print('Количество параметров', sum(np.product(t.shape) for t in sentence_level_model.parameters()))

# (histories[exp_name]['train_history'],
#  histories[exp_name]['best_model']) = train_eval_loop(sentence_level_model,
#                                               train_dataset,
#                                               test_dataset,
#                                               F.cross_entropy,
#                                               lr=hyps['lr'],
#                                               epoch_n=hyps['epoch_n'],
#                                               batch_size=hyps['batch'],
#                                               device=hyps['device'],
#                                               early_stopping_patience=hyps['stop_pat'],
#                                               max_batches_per_epoch_train=500,
#                                               max_batches_per_epoch_val=300,
#                                               experiment_name = exp_name,
#                                               lr_scheduler_ctor=lambda optim: torch.optim.lr_scheduler.ReduceLROnPlateau(optim, patience=hyps['lr_sched_pat'],
#                                                                                                                          factor=hyps['lr_sched_fac'],
#                                                                                                                          verbose=True))

In [ ]:
# # Если Вы запускаете ноутбук на colab или kaggle, добавьте в начало пути ./stepik-dl-nlp
# torch.save(histories[exp_name]['best_model'].state_dict(), './stepik-dl-nlp/models/sentence_level_pos.pth')

In [ ]:
# # Если Вы запускаете ноутбук на colab или kaggle, добавьте в начало пути ./stepik-dl-nlp
# histories[exp_name]['best_model'].load_state_dict(torch.load('./stepik-dl-nlp/models/sentence_level_pos.pth'))

## Свёрточный модуль своими руками

In [ ]:
class MyConv1d(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, padding=0, dilation=1):
        #dilation not released, add to parameters only for no error
        super().__init__()
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.kernel_size = kernel_size
        self.padding = padding
        self.weight = nn.Parameter(torch.randn(in_channels * kernel_size, out_channels) / (in_channels * kernel_size),
                                   requires_grad=True)
        self.bias = nn.Parameter(torch.zeros(out_channels), requires_grad=True)
    
    def forward(self, x):
        """x - BatchSize x InChannels x SequenceLen"""

        batch_size, src_channels, sequence_len = x.shape        
        if self.padding > 0:
            pad = x.new_zeros(batch_size, src_channels, self.padding)
            x = torch.cat((pad, x, pad), dim=-1)
            sequence_len = x.shape[-1]

        chunks = []
        chunk_size = sequence_len - self.kernel_size + 1
        for offset in range(self.kernel_size):
            chunks.append(x[:, :, offset:offset + chunk_size])

        in_features = torch.cat(chunks, dim=1)  # BatchSize x InChannels * KernelSize x ChunkSize
        in_features = in_features.permute(0, 2, 1)  # BatchSize x ChunkSize x InChannels * KernelSize
        out_features = torch.bmm(in_features, self.weight.unsqueeze(0).expand(batch_size, -1, -1)) + self.bias.unsqueeze(0).unsqueeze(0)
        out_features = out_features.permute(0, 2, 1)  # BatchSize x OutChannels x ChunkSize
        return out_features

In [ ]:
# #hyperparameters
# exp_name = 'baseline_MyConv1d'
# hyps = {
#     'dataset': 'base',
#     'char_min_count': 5,
#     'char_max_doc_freq': 1.0, 
#     'MAX_SENT_LEN': MAX_SENT_LEN,
#     'MAX_ORIG_TOKEN_LEN': MAX_ORIG_TOKEN_LEN,
    
#     #TokenPOSTagger
#     'emb_size': 64,
#     'sng_layers_n': 3, 
#     'sng_kernel_size': 3, 
#     'sng_dropout': 0.3,
#     'sng_conv_layer': MyConv1d,
#     'ctx_layers_n': 3, 
#     'ctx_kernel_size': 3, 
#     'ctx_dropout': 0.3,
#     'ctx_conv_layer': MyConv1d,

#     #train_eval_loop
#     'lr': 5e-3,
#     'epoch_n': 30,
#     'batch': 64,
#     'device': 'cuda',
#     'stop_pat': 5,
#     'lr_sched_pat': 2,
#     'lr_sched_fac': 0.5
# }

# histories[exp_name] = {
#     'hyps': hyps
# }

# sentence_level_model = SentenceLevelPOSTagger(len(char_vocab), len(label2id), embedding_size=hyps['emb_size'],
#                                               single_backbone_kwargs=dict(layers_n=hyps['sng_layers_n'], kernel_size=hyps['sng_kernel_size'], 
#                                                                           dropout=hyps['sng_dropout'], conv_layer=hyps['sng_conv_layer']),
#                                               context_backbone_kwargs=dict(layers_n=hyps['ctx_layers_n'], kernel_size=hyps['ctx_kernel_size'], 
#                                                                            dropout=hyps['ctx_dropout'], conv_layer=hyps['ctx_conv_layer']))
# print('Количество параметров', sum(np.product(t.shape) for t in sentence_level_model.parameters()))

# (histories[exp_name]['train_history'],
#  histories[exp_name]['best_model']) = train_eval_loop(sentence_level_model,
#                                               train_dataset,
#                                               test_dataset,
#                                               F.cross_entropy,
#                                               lr=hyps['lr'],
#                                               epoch_n=hyps['epoch_n'],
#                                               batch_size=hyps['batch'],
#                                               device=hyps['device'],
#                                               early_stopping_patience=hyps['stop_pat'],
#                                               max_batches_per_epoch_train=500,
#                                               max_batches_per_epoch_val=300,
#                                               experiment_name = exp_name,
#                                               lr_scheduler_ctor=lambda optim: torch.optim.lr_scheduler.ReduceLROnPlateau(optim, patience=hyps['lr_sched_pat'],
#                                                                                                                          factor=hyps['lr_sched_fac'],
#                                                                                                                          verbose=True))

## Финальное задание 3.7.Final

### Эксперимент - переписать код миниреснета на структуру без циклов

In [ ]:
class wisResidualBlock(torch.nn.Module):
    #v1.0
    def __init__(self, features_num, kernel_size=3, conv_layer=nn.Conv1d, dropout=0.0, batchnorm = None):
        #batchnorm = None, nn.InstanceNorm1d, nn.BatchNorm1d
        
        super().__init__()
        if batchnorm is None:
            self.layer = nn.Sequential(
                conv_layer(features_num, features_num, kernel_size, padding=kernel_size//2),
                nn.Dropout(dropout),
                nn.LeakyReLU())
        else:
            self.layer = nn.Sequential(
                conv_layer(features_num, features_num, kernel_size, padding=kernel_size//2),
                batchnorm(features_num),
                nn.Dropout(dropout),
                nn.LeakyReLU())
            
    def forward(self, x):        
        return x + self.layer(x)

class wisResidualBlock_act_out(torch.nn.Module):
    #v1.0
    def __init__(self, features_num, kernel_size=3, conv_layer=nn.Conv1d, dropout=0.0, batchnorm = None):
        #batchnorm = None, nn.InstanceNorm1d, nn.BatchNorm1d
        
        super().__init__()
        if batchnorm is None:
            self.layer = nn.Sequential(
                conv_layer(features_num, features_num, kernel_size, padding=kernel_size//2),
                nn.Dropout(dropout))
            self.act = nn.LeakyReLU()
        else:
            self.layer = nn.Sequential(
                conv_layer(features_num, features_num, kernel_size, padding=kernel_size//2),
                batchnorm(features_num),
                nn.Dropout(dropout),
                nn.LeakyReLU())
            
    def forward(self, x):
        return self.act(x + self.layer(x))

In [ ]:
class wisStackedConv1d(nn.Module):
    #v1.0
    def __init__(self, features_num, layers_n=1, res_block = wisResidualBlock, 
                 kernel_size=3, conv_layer=nn.Conv1d, dropout=0.0, batchnorm = None):
        super().__init__()
        self.layers = nn.Sequential(*[res_block(features_num, 
                                                kernel_size=kernel_size, 
                                                conv_layer=conv_layer,
                                                dropout=dropout,
                                                batchnorm=batchnorm) for _ in range(layers_n)])
    
    def forward(self, x):
        """x - BatchSize x FeaturesNum x SequenceLen"""
        return self.layers(x)

In [ ]:
class SentenceLevelPOSTagger(nn.Module):
    #v1.0
    def __init__(self, vocab_size, labels_num, embedding_size=32, res_block = wisResidualBlock, 
                 single_backbone_kwargs={}, context_backbone_kwargs={}):
        super().__init__()
        self.embedding_size = embedding_size
        self.char_embeddings = nn.Embedding(vocab_size, embedding_size, padding_idx=0)
        self.single_token_backbone = wisStackedConv1d(embedding_size, res_block=res_block, **single_backbone_kwargs)
        self.context_backbone = wisStackedConv1d(embedding_size, res_block=res_block, **context_backbone_kwargs)
        self.global_pooling = nn.AdaptiveMaxPool1d(1)
        self.out = nn.Conv1d(embedding_size, labels_num, 1)
        self.labels_num = labels_num
    
    def forward(self, tokens):
        """tokens - BatchSize x MaxSentenceLen x MaxTokenLen"""
        batch_size, max_sent_len, max_token_len = tokens.shape
        tokens_flat = tokens.view(batch_size * max_sent_len, max_token_len)
        
        char_embeddings = self.char_embeddings(tokens_flat)  # BatchSize*MaxSentenceLen x MaxTokenLen x EmbSize
        char_embeddings = char_embeddings.permute(0, 2, 1)  # BatchSize*MaxSentenceLen x EmbSize x MaxTokenLen
        char_features = self.single_token_backbone(char_embeddings)
        
        token_features_flat = self.global_pooling(char_features).squeeze(-1)  # BatchSize*MaxSentenceLen x EmbSize

        token_features = token_features_flat.view(batch_size, max_sent_len, self.embedding_size)  # BatchSize x MaxSentenceLen x EmbSize
        token_features = token_features.permute(0, 2, 1)  # BatchSize x EmbSize x MaxSentenceLen
        context_features = self.context_backbone(token_features)  # BatchSize x EmbSize x MaxSentenceLen

        logits = self.out(context_features)  # BatchSize x LabelsNum x MaxSentenceLen
        return logits

In [ ]:
# #hyperparameters
# exp_name = 'wisResidualBlock_MyConv1d_01'
# hyps = {
#     'dataset': 'base',
#     'char_min_count': 5,
#     'char_max_doc_freq': 1.0, 
#     'MAX_SENT_LEN': MAX_SENT_LEN,
#     'MAX_ORIG_TOKEN_LEN': MAX_ORIG_TOKEN_LEN,
    
#     #TokenPOSTagger
#     'emb_size': 64,
#     'res_block': wisResidualBlock,
#     'sng_backbone_kwargs': {
#         'layers_n': 3, 
#         'kernel_size': 3, 
#         'dropout': 0.3,
#         'conv_layer': MyConv1d,
#         'batchnorm': nn.BatchNorm1d },
#     'ctx_backbone_kwargs': {
#         'layers_n': 3, 
#         'kernel_size': 3, 
#         'dropout': 0.3,
#         'conv_layer': MyConv1d,
#         'batchnorm': nn.BatchNorm1d },

#     #train_eval_loop
#     'lr': 5e-3,
#     'epoch_n': 1,
#     'batch': 64,
#     'device': 'cuda',
#     'stop_pat': 5,
#     'lr_sched_pat': 2,
#     'lr_sched_fac': 0.5
# }

# histories[exp_name] = {
#     'hyps': hyps
# }

# sentence_level_model = SentenceLevelPOSTagger(len(char_vocab), len(label2id), 
#                                               embedding_size=hyps['emb_size'], res_block=hyps['res_block'],
#                                               single_backbone_kwargs=hyps['sng_backbone_kwargs'],
#                                               context_backbone_kwargs=hyps['ctx_backbone_kwargs'])
# print('Количество параметров', sum(np.product(t.shape) for t in sentence_level_model.parameters()))

# (histories[exp_name]['train_history'],
#  histories[exp_name]['best_model']) = train_eval_loop(sentence_level_model,
#                                               train_dataset,
#                                               test_dataset,
#                                               F.cross_entropy,
#                                               lr=hyps['lr'],
#                                               epoch_n=hyps['epoch_n'],
#                                               batch_size=hyps['batch'],
#                                               device=hyps['device'],
#                                               early_stopping_patience=hyps['stop_pat'],
#                                               max_batches_per_epoch_train=500,
#                                               max_batches_per_epoch_val=300,
#                                               best_acc_type = 'acc',
#                                               experiment_name = exp_name,
#                                               lr_scheduler_ctor=lambda optim: torch.optim.lr_scheduler.ReduceLROnPlateau(optim, patience=hyps['lr_sched_pat'],
#                                                                                                                          factor=hyps['lr_sched_fac'],
#                                                                                                                          verbose=True))

In [ ]:
# #hyperparameters
# exp_name = 'wisResidualBlock_MyConv1d_01'
# hyps = {
#     'dataset': 'base',
#     'char_min_count': 5,
#     'char_max_doc_freq': 1.0, 
#     'MAX_SENT_LEN': MAX_SENT_LEN,
#     'MAX_ORIG_TOKEN_LEN': MAX_ORIG_TOKEN_LEN,
    
#     #TokenPOSTagger
#     'emb_size': 64,
#     'res_block': wisResidualBlock,
#     'sng_backbone_kwargs': {
#         'layers_n': 3, 
#         'kernel_size': 3, 
#         'dropout': 0.3,
#         'conv_layer': MyConv1d,
#         'batchnorm': nn.InstanceNorm1d },
#     'ctx_backbone_kwargs': {
#         'layers_n': 3, 
#         'kernel_size': 3, 
#         'dropout': 0.3,
#         'conv_layer': MyConv1d,
#         'batchnorm': nn.InstanceNorm1d },

#     #train_eval_loop
#     'lr': 5e-3,
#     'epoch_n': 1,
#     'batch': 64,
#     'device': 'cuda',
#     'stop_pat': 5,
#     'lr_sched_pat': 2,
#     'lr_sched_fac': 0.5
# }

# histories[exp_name] = {
#     'hyps': hyps
# }

# sentence_level_model = SentenceLevelPOSTagger(len(char_vocab), len(label2id), 
#                                               embedding_size=hyps['emb_size'], res_block=hyps['res_block'],
#                                               single_backbone_kwargs=hyps['sng_backbone_kwargs'],
#                                               context_backbone_kwargs=hyps['ctx_backbone_kwargs'])
# print('Количество параметров', sum(np.product(t.shape) for t in sentence_level_model.parameters()))

# (histories[exp_name]['train_history'],
#  histories[exp_name]['best_model']) = train_eval_loop(sentence_level_model,
#                                               train_dataset,
#                                               test_dataset,
#                                               F.cross_entropy,
#                                               lr=hyps['lr'],
#                                               epoch_n=hyps['epoch_n'],
#                                               batch_size=hyps['batch'],
#                                               device=hyps['device'],
#                                               early_stopping_patience=hyps['stop_pat'],
#                                               max_batches_per_epoch_train=500,
#                                               max_batches_per_epoch_val=300,
#                                               best_acc_type = 'loss',
#                                               experiment_name = exp_name,
#                                               lr_scheduler_ctor=lambda optim: torch.optim.lr_scheduler.ReduceLROnPlateau(optim, patience=hyps['lr_sched_pat'],
#                                                                                                                          factor=hyps['lr_sched_fac'],
#                                                                                                                          verbose=True))

In [ ]:
# #hyperparameters
# exp_name = 'wisResidualBlock_MyConv1d_01'
# hyps = {
#     'dataset': 'base',
#     'char_min_count': 5,
#     'char_max_doc_freq': 1.0, 
#     'MAX_SENT_LEN': MAX_SENT_LEN,
#     'MAX_ORIG_TOKEN_LEN': MAX_ORIG_TOKEN_LEN,
    
#     #TokenPOSTagger
#     'emb_size': 64,
#     'res_block': wisResidualBlock,
#     'sng_layers_n': 3, 
#     'sng_kernel_size': 3, 
#     'sng_dropout': 0.3,
#     'sng_conv_layer': MyConv1d,
#     'ctx_layers_n': 3, 
#     'ctx_kernel_size': 3, 
#     'ctx_dropout': 0.3,
#     'ctx_conv_layer': MyConv1d,

#     #train_eval_loop
#     'lr': 5e-3,
#     'epoch_n': 40,
#     'batch': 64,
#     'device': 'cuda',
#     'stop_pat': 5,
#     'lr_sched_pat': 2,
#     'lr_sched_fac': 0.5
# }

# histories[exp_name] = {
#     'hyps': hyps
# }

# sentence_level_model = SentenceLevelPOSTagger(len(char_vocab), len(label2id), 
#                                               embedding_size=hyps['emb_size'], res_block=hyps['res_block'],
#                                               single_backbone_kwargs=dict(layers_n=hyps['sng_layers_n'],
#                                                                           kernel_size=hyps['sng_kernel_size'],
#                                                                           dropout=hyps['sng_dropout'],
#                                                                           conv_layer=hyps['sng_conv_layer']),
#                                               context_backbone_kwargs=dict(layers_n=hyps['ctx_layers_n'],
#                                                                            kernel_size=hyps['ctx_kernel_size'],
#                                                                            dropout=hyps['ctx_dropout'],
#                                                                            conv_layer=hyps['ctx_conv_layer']))
# print('Количество параметров', sum(np.product(t.shape) for t in sentence_level_model.parameters()))

# (histories[exp_name]['train_history'],
#  histories[exp_name]['best_model']) = train_eval_loop(sentence_level_model,
#                                               train_dataset,
#                                               test_dataset,
#                                               F.cross_entropy,
#                                               lr=hyps['lr'],
#                                               epoch_n=hyps['epoch_n'],
#                                               batch_size=hyps['batch'],
#                                               device=hyps['device'],
#                                               early_stopping_patience=hyps['stop_pat'],
#                                               max_batches_per_epoch_train=500,
#                                               max_batches_per_epoch_val=300,
#                                               best_acc_type = 'loss',
#                                               experiment_name = exp_name,
#                                               lr_scheduler_ctor=lambda optim: torch.optim.lr_scheduler.ReduceLROnPlateau(optim, patience=hyps['lr_sched_pat'],
#                                                                                                                          factor=hyps['lr_sched_fac'],
#                                                                                                                          verbose=True))

In [ ]:
# #hyperparameters
# exp_name = 'wisResidualBlock_MyConv1d_02'
# hyps = {
#     'dataset': 'base',
#     'char_min_count': 5,
#     'char_max_doc_freq': 1.0, 
#     'MAX_SENT_LEN': MAX_SENT_LEN,
#     'MAX_ORIG_TOKEN_LEN': MAX_ORIG_TOKEN_LEN,
    
#     #TokenPOSTagger
#     'emb_size': 64,
#     'res_block': wisResidualBlock,
#     'sng_layers_n': 3, 
#     'sng_kernel_size': 3, 
#     'sng_dropout': 0.3,
#     'sng_conv_layer': MyConv1d,
#     'ctx_layers_n': 3, 
#     'ctx_kernel_size': 3, 
#     'ctx_dropout': 0.3,
#     'ctx_conv_layer': MyConv1d,

#     #train_eval_loop
#     'lr': 5e-3,
#     'epoch_n': 30,
#     'batch': 64,
#     'device': 'cuda',
#     'stop_pat': 5,
#     'lr_sched_pat': 2,
#     'lr_sched_fac': 0.5
# }

# histories[exp_name] = {
#     'hyps': hyps
# }

# sentence_level_model = SentenceLevelPOSTagger(len(char_vocab), len(label2id), 
#                                               embedding_size=hyps['emb_size'], res_block=hyps['res_block'],
#                                               single_backbone_kwargs=dict(layers_n=hyps['sng_layers_n'],
#                                                                           kernel_size=hyps['sng_kernel_size'],
#                                                                           dropout=hyps['sng_dropout'],
#                                                                           conv_layer=hyps['sng_conv_layer']),
#                                               context_backbone_kwargs=dict(layers_n=hyps['ctx_layers_n'],
#                                                                            kernel_size=hyps['ctx_kernel_size'],
#                                                                            dropout=hyps['ctx_dropout'],
#                                                                            conv_layer=hyps['ctx_conv_layer']))
# print('Количество параметров', sum(np.product(t.shape) for t in sentence_level_model.parameters()))

# (histories[exp_name]['train_history'],
#  histories[exp_name]['best_model']) = train_eval_loop(sentence_level_model,
#                                               train_dataset,
#                                               test_dataset,
#                                               F.cross_entropy,
#                                               lr=hyps['lr'],
#                                               epoch_n=hyps['epoch_n'],
#                                               batch_size=hyps['batch'],
#                                               device=hyps['device'],
#                                               early_stopping_patience=hyps['stop_pat'],
#                                               max_batches_per_epoch_train=500,
#                                               max_batches_per_epoch_val=300,
#                                               best_acc_type = 'loss',
#                                               experiment_name = exp_name,
#                                               lr_scheduler_ctor=lambda optim: torch.optim.lr_scheduler.ReduceLROnPlateau(optim, patience=hyps['lr_sched_pat'],
#                                                                                                                          factor=hyps['lr_sched_fac'],
#                                                                                                                          verbose=True))

In [ ]:
# #hyperparameters
# exp_name = 'baseline_wisResidualBlock_act_out_MyConv1d_01'
# hyps = {
#     'dataset': 'base',
#     'char_min_count': 5,
#     'char_max_doc_freq': 1.0, 
#     'MAX_SENT_LEN': MAX_SENT_LEN,
#     'MAX_ORIG_TOKEN_LEN': MAX_ORIG_TOKEN_LEN,
    
#     #TokenPOSTagger
#     'emb_size': 64,
#     'res_block': wisResidualBlock_act_out,
#     'sng_layers_n': 3, 
#     'sng_kernel_size': 3, 
#     'sng_dropout': 0.3,
#     'sng_conv_layer': MyConv1d,
#     'ctx_layers_n': 3, 
#     'ctx_kernel_size': 3, 
#     'ctx_dropout': 0.3,
#     'ctx_conv_layer': MyConv1d,

#     #train_eval_loop
#     'lr': 5e-3,
#     'epoch_n': 40,
#     'batch': 64,
#     'device': 'cuda',
#     'stop_pat': 5,
#     'lr_sched_pat': 2,
#     'lr_sched_fac': 0.5
# }

# histories[exp_name] = {
#     'hyps': hyps
# }

# sentence_level_model = SentenceLevelPOSTagger(len(char_vocab), len(label2id), 
#                                               embedding_size=hyps['emb_size'], res_block=hyps['res_block'],
#                                               single_backbone_kwargs=dict(layers_n=hyps['sng_layers_n'],
#                                                                           kernel_size=hyps['sng_kernel_size'],
#                                                                           dropout=hyps['sng_dropout'],
#                                                                           conv_layer=hyps['sng_conv_layer']),
#                                               context_backbone_kwargs=dict(layers_n=hyps['ctx_layers_n'],
#                                                                            kernel_size=hyps['ctx_kernel_size'],
#                                                                            dropout=hyps['ctx_dropout'],
#                                                                            conv_layer=hyps['ctx_conv_layer']))
# print('Количество параметров', sum(np.product(t.shape) for t in sentence_level_model.parameters()))

# (histories[exp_name]['train_history'],
#  histories[exp_name]['best_model']) = train_eval_loop(sentence_level_model,
#                                               train_dataset,
#                                               test_dataset,
#                                               F.cross_entropy,
#                                               lr=hyps['lr'],
#                                               epoch_n=hyps['epoch_n'],
#                                               batch_size=hyps['batch'],
#                                               device=hyps['device'],
#                                               early_stopping_patience=hyps['stop_pat'],
#                                               max_batches_per_epoch_train=500,
#                                               max_batches_per_epoch_val=300,
#                                               best_acc_type = 'loss',
#                                               experiment_name = exp_name,
#                                               lr_scheduler_ctor=lambda optim: torch.optim.lr_scheduler.ReduceLROnPlateau(optim, patience=hyps['lr_sched_pat'],
#                                                                                                                          factor=hyps['lr_sched_fac'],
#                                                                                                                          verbose=True))

In [ ]:
# #hyperparameters
# exp_name = 'baseline_wisResidualBlock_act_out_MyConv1d_02'
# hyps = {
#     'dataset': 'base',
#     'char_min_count': 5,
#     'char_max_doc_freq': 1.0, 
#     'MAX_SENT_LEN': MAX_SENT_LEN,
#     'MAX_ORIG_TOKEN_LEN': MAX_ORIG_TOKEN_LEN,
    
#     #TokenPOSTagger
#     'emb_size': 64,
#     'res_block': wisResidualBlock_act_out,
#     'sng_layers_n': 3, 
#     'sng_kernel_size': 3, 
#     'sng_dropout': 0.3,
#     'sng_conv_layer': MyConv1d,
#     'ctx_layers_n': 3, 
#     'ctx_kernel_size': 3, 
#     'ctx_dropout': 0.3,
#     'ctx_conv_layer': MyConv1d,

#     #train_eval_loop
#     'lr': 5e-3,
#     'epoch_n': 30,
#     'batch': 64,
#     'device': 'cuda',
#     'stop_pat': 5,
#     'lr_sched_pat': 2,
#     'lr_sched_fac': 0.5
# }

# histories[exp_name] = {
#     'hyps': hyps
# }

# sentence_level_model = SentenceLevelPOSTagger(len(char_vocab), len(label2id), 
#                                               embedding_size=hyps['emb_size'], res_block=hyps['res_block'],
#                                               single_backbone_kwargs=dict(layers_n=hyps['sng_layers_n'],
#                                                                           kernel_size=hyps['sng_kernel_size'],
#                                                                           dropout=hyps['sng_dropout'],
#                                                                           conv_layer=hyps['sng_conv_layer']),
#                                               context_backbone_kwargs=dict(layers_n=hyps['ctx_layers_n'],
#                                                                            kernel_size=hyps['ctx_kernel_size'],
#                                                                            dropout=hyps['ctx_dropout'],
#                                                                            conv_layer=hyps['ctx_conv_layer']))
# print('Количество параметров', sum(np.product(t.shape) for t in sentence_level_model.parameters()))

# (histories[exp_name]['train_history'],
#  histories[exp_name]['best_model']) = train_eval_loop(sentence_level_model,
#                                               train_dataset,
#                                               test_dataset,
#                                               F.cross_entropy,
#                                               lr=hyps['lr'],
#                                               epoch_n=hyps['epoch_n'],
#                                               batch_size=hyps['batch'],
#                                               device=hyps['device'],
#                                               early_stopping_patience=hyps['stop_pat'],
#                                               max_batches_per_epoch_train=500,
#                                               max_batches_per_epoch_val=300,
#                                               best_acc_type = 'loss',
#                                               experiment_name = exp_name,
#                                               lr_scheduler_ctor=lambda optim: torch.optim.lr_scheduler.ReduceLROnPlateau(optim, patience=hyps['lr_sched_pat'],
#                                                                                                                          factor=hyps['lr_sched_fac'],
#                                                                                                                          verbose=True))

### Эксперимент - поиграться с параметрами и архитектурой - количеством каналов (размерностью эмбеддинга), глубиной нейросети, силой Dropout, добавить BatchNorm или другую нормализацию

In [ ]:
# #hyperparameters
# exp_name = 'baseline'
# hyps = {
#     'dataset': 'base',
#     'char_min_count': 5,
#     'char_max_doc_freq': 1.0, 
#     'MAX_SENT_LEN': MAX_SENT_LEN,
#     'MAX_ORIG_TOKEN_LEN': MAX_ORIG_TOKEN_LEN,
    
#     #TokenPOSTagger
#     'emb_size': 64,
#     'res_block': wisResidualBlock,
#     'sng_backbone_kwargs': {
#         'layers_n': 3, 
#         'kernel_size': 3, 
#         'dropout': 0.3,
#         'conv_layer': MyConv1d,
#         'batchnorm': None },
#     'ctx_backbone_kwargs': {
#         'layers_n': 3, 
#         'kernel_size': 3, 
#         'dropout': 0.3,
#         'conv_layer': MyConv1d,
#         'batchnorm': None },

#     #train_eval_loop
#     'lr': 5e-3,
#     'epoch_n': 30,
#     'batch': 64,
#     'device': 'cuda',
#     'stop_pat': 5,
#     'lr_sched_pat': 2,
#     'lr_sched_fac': 0.5
# }

# histories[exp_name] = {
#     'hyps': hyps
# }


# sentence_level_model = SentenceLevelPOSTagger(len(char_vocab), len(label2id), 
#                                               embedding_size=hyps['emb_size'], res_block=hyps['res_block'],
#                                               single_backbone_kwargs=hyps['sng_backbone_kwargs'],
#                                               context_backbone_kwargs=hyps['ctx_backbone_kwargs'])
# print('Количество параметров', sum(np.product(t.shape) for t in sentence_level_model.parameters()))

# (histories[exp_name]['train_history'],
#  histories[exp_name]['best_model']) = train_eval_loop(sentence_level_model,
#                                               train_dataset,
#                                               test_dataset,
#                                               F.cross_entropy,
#                                               lr=hyps['lr'],
#                                               epoch_n=hyps['epoch_n'],
#                                               batch_size=hyps['batch'],
#                                               device=hyps['device'],
#                                               early_stopping_patience=hyps['stop_pat'],
#                                               max_batches_per_epoch_train=500,
#                                               max_batches_per_epoch_val=300,
#                                               best_acc_type = 'acc',
#                                               experiment_name = exp_name,
#                                               lr_scheduler_ctor=lambda optim: torch.optim.lr_scheduler.ReduceLROnPlateau(optim, patience=hyps['lr_sched_pat'],
#                                                                                                                          factor=hyps['lr_sched_fac'],
#                                                                                                                          verbose=True))

In [ ]:
# #hyperparameters
# exp_name = 'lr=1e-2'
# hyps = {
#     'dataset': 'base',
#     'char_min_count': 5,
#     'char_max_doc_freq': 1.0, 
#     'MAX_SENT_LEN': MAX_SENT_LEN,
#     'MAX_ORIG_TOKEN_LEN': MAX_ORIG_TOKEN_LEN,
    
#     #TokenPOSTagger
#     'emb_size': 64,
#     'res_block': wisResidualBlock,
#     'sng_backbone_kwargs': {
#         'layers_n': 3, 
#         'kernel_size': 3, 
#         'dropout': 0.3,
#         'conv_layer': MyConv1d,
#         'batchnorm': None },
#     'ctx_backbone_kwargs': {
#         'layers_n': 3, 
#         'kernel_size': 3, 
#         'dropout': 0.3,
#         'conv_layer': MyConv1d,
#         'batchnorm': None },

#     #train_eval_loop
#     'lr': 1e-2,
#     'epoch_n': 30,
#     'batch': 64,
#     'device': 'cuda',
#     'stop_pat': 5,
#     'lr_sched_pat': 2,
#     'lr_sched_fac': 0.5
# }

# histories[exp_name] = {
#     'hyps': hyps
# }

# sentence_level_model = SentenceLevelPOSTagger(len(char_vocab), len(label2id), 
#                                               embedding_size=hyps['emb_size'], res_block=hyps['res_block'],
#                                               single_backbone_kwargs=hyps['sng_backbone_kwargs'],
#                                               context_backbone_kwargs=hyps['ctx_backbone_kwargs'])
# print('Количество параметров', sum(np.product(t.shape) for t in sentence_level_model.parameters()))

# (histories[exp_name]['train_history'],
#  histories[exp_name]['best_model']) = train_eval_loop(sentence_level_model,
#                                               train_dataset,
#                                               test_dataset,
#                                               F.cross_entropy,
#                                               lr=hyps['lr'],
#                                               epoch_n=hyps['epoch_n'],
#                                               batch_size=hyps['batch'],
#                                               device=hyps['device'],
#                                               early_stopping_patience=hyps['stop_pat'],
#                                               max_batches_per_epoch_train=500,
#                                               max_batches_per_epoch_val=300,
#                                               best_acc_type = 'acc',
#                                               experiment_name = exp_name,
#                                               lr_scheduler_ctor=lambda optim: torch.optim.lr_scheduler.ReduceLROnPlateau(optim, patience=hyps['lr_sched_pat'],
#                                                                                                                          factor=hyps['lr_sched_fac'],
#                                                                                                                          verbose=True))

In [ ]:
# #hyperparameters
# exp_name = 'emb_size=100'
# hyps = {
#     'dataset': 'base',
#     'char_min_count': 5,
#     'char_max_doc_freq': 1.0, 
#     'MAX_SENT_LEN': MAX_SENT_LEN,
#     'MAX_ORIG_TOKEN_LEN': MAX_ORIG_TOKEN_LEN,
    
#     #TokenPOSTagger
#     'emb_size': 100,
#     'res_block': wisResidualBlock,
#     'sng_backbone_kwargs': {
#         'layers_n': 3, 
#         'kernel_size': 3, 
#         'dropout': 0.3,
#         'conv_layer': MyConv1d,
#         'batchnorm': None },
#     'ctx_backbone_kwargs': {
#         'layers_n': 3, 
#         'kernel_size': 3, 
#         'dropout': 0.3,
#         'conv_layer': MyConv1d,
#         'batchnorm': None },

#     #train_eval_loop
#     'lr': 5e-3,
#     'epoch_n': 30,
#     'batch': 64,
#     'device': 'cuda',
#     'stop_pat': 5,
#     'lr_sched_pat': 2,
#     'lr_sched_fac': 0.5
# }

# histories[exp_name] = {
#     'hyps': hyps
# }

# sentence_level_model = SentenceLevelPOSTagger(len(char_vocab), len(label2id), 
#                                               embedding_size=hyps['emb_size'], res_block=hyps['res_block'],
#                                               single_backbone_kwargs=hyps['sng_backbone_kwargs'],
#                                               context_backbone_kwargs=hyps['ctx_backbone_kwargs'])
# print('Количество параметров', sum(np.product(t.shape) for t in sentence_level_model.parameters()))

# (histories[exp_name]['train_history'],
#  histories[exp_name]['best_model']) = train_eval_loop(sentence_level_model,
#                                               train_dataset,
#                                               test_dataset,
#                                               F.cross_entropy,
#                                               lr=hyps['lr'],
#                                               epoch_n=hyps['epoch_n'],
#                                               batch_size=hyps['batch'],
#                                               device=hyps['device'],
#                                               early_stopping_patience=hyps['stop_pat'],
#                                               max_batches_per_epoch_train=500,
#                                               max_batches_per_epoch_val=300,
#                                               best_acc_type = 'acc',
#                                               experiment_name = exp_name,
#                                               lr_scheduler_ctor=lambda optim: torch.optim.lr_scheduler.ReduceLROnPlateau(optim, patience=hyps['lr_sched_pat'],
#                                                                                                                          factor=hyps['lr_sched_fac'],
#                                                                                                                          verbose=True))

In [ ]:
# #hyperparameters
# exp_name = 'emb_size=150'
# hyps = {
#     'dataset': 'base',
#     'char_min_count': 5,
#     'char_max_doc_freq': 1.0, 
#     'MAX_SENT_LEN': MAX_SENT_LEN,
#     'MAX_ORIG_TOKEN_LEN': MAX_ORIG_TOKEN_LEN,
    
#     #TokenPOSTagger
#     'emb_size': 150,
#     'res_block': wisResidualBlock,
#     'sng_backbone_kwargs': {
#         'layers_n': 3, 
#         'kernel_size': 3, 
#         'dropout': 0.3,
#         'conv_layer': MyConv1d,
#         'batchnorm': None },
#     'ctx_backbone_kwargs': {
#         'layers_n': 3, 
#         'kernel_size': 3, 
#         'dropout': 0.3,
#         'conv_layer': MyConv1d,
#         'batchnorm': None },

#     #train_eval_loop
#     'lr': 5e-3,
#     'epoch_n': 30,
#     'batch': 64,
#     'device': 'cuda',
#     'stop_pat': 5,
#     'lr_sched_pat': 2,
#     'lr_sched_fac': 0.5
# }

# histories[exp_name] = {
#     'hyps': hyps
# }

# sentence_level_model = SentenceLevelPOSTagger(len(char_vocab), len(label2id), 
#                                               embedding_size=hyps['emb_size'], res_block=hyps['res_block'],
#                                               single_backbone_kwargs=hyps['sng_backbone_kwargs'],
#                                               context_backbone_kwargs=hyps['ctx_backbone_kwargs'])
# print('Количество параметров', sum(np.product(t.shape) for t in sentence_level_model.parameters()))

# (histories[exp_name]['train_history'],
#  histories[exp_name]['best_model']) = train_eval_loop(sentence_level_model,
#                                               train_dataset,
#                                               test_dataset,
#                                               F.cross_entropy,
#                                               lr=hyps['lr'],
#                                               epoch_n=hyps['epoch_n'],
#                                               batch_size=hyps['batch'],
#                                               device=hyps['device'],
#                                               early_stopping_patience=hyps['stop_pat'],
#                                               max_batches_per_epoch_train=500,
#                                               max_batches_per_epoch_val=300,
#                                               best_acc_type = 'acc',
#                                               experiment_name = exp_name,
#                                               lr_scheduler_ctor=lambda optim: torch.optim.lr_scheduler.ReduceLROnPlateau(optim, patience=hyps['lr_sched_pat'],
#                                                                                                                          factor=hyps['lr_sched_fac'],
#                                                                                                                          verbose=True))

In [ ]:
# #hyperparameters
# exp_name = 'dropout=0.1'
# hyps = {
#     'dataset': 'base',
#     'char_min_count': 5,
#     'char_max_doc_freq': 1.0, 
#     'MAX_SENT_LEN': MAX_SENT_LEN,
#     'MAX_ORIG_TOKEN_LEN': MAX_ORIG_TOKEN_LEN,
    
#     #TokenPOSTagger
#     'emb_size': 64,
#     'res_block': wisResidualBlock,
#     'sng_backbone_kwargs': {
#         'layers_n': 3, 
#         'kernel_size': 3, 
#         'dropout': 0.1,
#         'conv_layer': MyConv1d,
#         'batchnorm': None },
#     'ctx_backbone_kwargs': {
#         'layers_n': 3, 
#         'kernel_size': 3, 
#         'dropout': 0.1,
#         'conv_layer': MyConv1d,
#         'batchnorm': None },

#     #train_eval_loop
#     'lr': 5e-3,
#     'epoch_n': 30,
#     'batch': 64,
#     'device': 'cuda',
#     'stop_pat': 5,
#     'lr_sched_pat': 2,
#     'lr_sched_fac': 0.5
# }

# histories[exp_name] = {
#     'hyps': hyps
# }

# sentence_level_model = SentenceLevelPOSTagger(len(char_vocab), len(label2id), 
#                                               embedding_size=hyps['emb_size'], res_block=hyps['res_block'],
#                                               single_backbone_kwargs=hyps['sng_backbone_kwargs'],
#                                               context_backbone_kwargs=hyps['ctx_backbone_kwargs'])
# print('Количество параметров', sum(np.product(t.shape) for t in sentence_level_model.parameters()))

# (histories[exp_name]['train_history'],
#  histories[exp_name]['best_model']) = train_eval_loop(sentence_level_model,
#                                               train_dataset,
#                                               test_dataset,
#                                               F.cross_entropy,
#                                               lr=hyps['lr'],
#                                               epoch_n=hyps['epoch_n'],
#                                               batch_size=hyps['batch'],
#                                               device=hyps['device'],
#                                               early_stopping_patience=hyps['stop_pat'],
#                                               max_batches_per_epoch_train=500,
#                                               max_batches_per_epoch_val=300,
#                                               best_acc_type = 'acc',
#                                               experiment_name = exp_name,
#                                               lr_scheduler_ctor=lambda optim: torch.optim.lr_scheduler.ReduceLROnPlateau(optim, patience=hyps['lr_sched_pat'],
#                                                                                                                          factor=hyps['lr_sched_fac'],
#                                                                                                                          verbose=True))

In [ ]:
# #hyperparameters
# exp_name = 'nn.BatchNorm1d'
# hyps = {
#     'dataset': 'base',
#     'char_min_count': 5,
#     'char_max_doc_freq': 1.0, 
#     'MAX_SENT_LEN': MAX_SENT_LEN,
#     'MAX_ORIG_TOKEN_LEN': MAX_ORIG_TOKEN_LEN,
    
#     #TokenPOSTagger
#     'emb_size': 64,
#     'res_block': wisResidualBlock,
#     'sng_backbone_kwargs': {
#         'layers_n': 3, 
#         'kernel_size': 3, 
#         'dropout': 0.3,
#         'conv_layer': MyConv1d,
#         'batchnorm': nn.BatchNorm1d },
#     'ctx_backbone_kwargs': {
#         'layers_n': 3, 
#         'kernel_size': 3, 
#         'dropout': 0.3,
#         'conv_layer': MyConv1d,
#         'batchnorm': nn.BatchNorm1d },

#     #train_eval_loop
#     'lr': 5e-3,
#     'epoch_n': 30,
#     'batch': 64,
#     'device': 'cuda',
#     'stop_pat': 5,
#     'lr_sched_pat': 2,
#     'lr_sched_fac': 0.5
# }

# histories[exp_name] = {
#     'hyps': hyps
# }

# sentence_level_model = SentenceLevelPOSTagger(len(char_vocab), len(label2id), 
#                                               embedding_size=hyps['emb_size'], res_block=hyps['res_block'],
#                                               single_backbone_kwargs=hyps['sng_backbone_kwargs'],
#                                               context_backbone_kwargs=hyps['ctx_backbone_kwargs'])
# print('Количество параметров', sum(np.product(t.shape) for t in sentence_level_model.parameters()))

# (histories[exp_name]['train_history'],
#  histories[exp_name]['best_model']) = train_eval_loop(sentence_level_model,
#                                               train_dataset,
#                                               test_dataset,
#                                               F.cross_entropy,
#                                               lr=hyps['lr'],
#                                               epoch_n=hyps['epoch_n'],
#                                               batch_size=hyps['batch'],
#                                               device=hyps['device'],
#                                               early_stopping_patience=hyps['stop_pat'],
#                                               max_batches_per_epoch_train=500,
#                                               max_batches_per_epoch_val=300,
#                                               best_acc_type = 'acc',
#                                               experiment_name = exp_name,
#                                               lr_scheduler_ctor=lambda optim: torch.optim.lr_scheduler.ReduceLROnPlateau(optim, patience=hyps['lr_sched_pat'],
#                                                                                                                          factor=hyps['lr_sched_fac'],
#                                                                                                                          verbose=True))

In [ ]:
# #hyperparameters
# exp_name = 'nn.InstanceNorm1d'
# hyps = {
#     'dataset': 'base',
#     'char_min_count': 5,
#     'char_max_doc_freq': 1.0, 
#     'MAX_SENT_LEN': MAX_SENT_LEN,
#     'MAX_ORIG_TOKEN_LEN': MAX_ORIG_TOKEN_LEN,
    
#     #TokenPOSTagger
#     'emb_size': 64,
#     'res_block': wisResidualBlock,
#     'sng_backbone_kwargs': {
#         'layers_n': 3, 
#         'kernel_size': 3, 
#         'dropout': 0.3,
#         'conv_layer': MyConv1d,
#         'batchnorm': nn.InstanceNorm1d },
#     'ctx_backbone_kwargs': {
#         'layers_n': 3, 
#         'kernel_size': 3, 
#         'dropout': 0.3,
#         'conv_layer': MyConv1d,
#         'batchnorm': nn.InstanceNorm1d },

#     #train_eval_loop
#     'lr': 5e-3,
#     'epoch_n': 30,
#     'batch': 64,
#     'device': 'cuda',
#     'stop_pat': 5,
#     'lr_sched_pat': 2,
#     'lr_sched_fac': 0.5
# }

# histories[exp_name] = {
#     'hyps': hyps
# }

# sentence_level_model = SentenceLevelPOSTagger(len(char_vocab), len(label2id), 
#                                               embedding_size=hyps['emb_size'], res_block=hyps['res_block'],
#                                               single_backbone_kwargs=hyps['sng_backbone_kwargs'],
#                                               context_backbone_kwargs=hyps['ctx_backbone_kwargs'])
# print('Количество параметров', sum(np.product(t.shape) for t in sentence_level_model.parameters()))

# (histories[exp_name]['train_history'],
#  histories[exp_name]['best_model']) = train_eval_loop(sentence_level_model,
#                                               train_dataset,
#                                               test_dataset,
#                                               F.cross_entropy,
#                                               lr=hyps['lr'],
#                                               epoch_n=hyps['epoch_n'],
#                                               batch_size=hyps['batch'],
#                                               device=hyps['device'],
#                                               early_stopping_patience=hyps['stop_pat'],
#                                               max_batches_per_epoch_train=500,
#                                               max_batches_per_epoch_val=300,
#                                               best_acc_type = 'acc',
#                                               experiment_name = exp_name,
#                                               lr_scheduler_ctor=lambda optim: torch.optim.lr_scheduler.ReduceLROnPlateau(optim, patience=hyps['lr_sched_pat'],
#                                                                                                                          factor=hyps['lr_sched_fac'],
#                                                                                                                          verbose=True))

In [ ]:
# #hyperparameters
# exp_name = 'sng_kernel_size=5'
# hyps = {
#     'dataset': 'base',
#     'char_min_count': 5,
#     'char_max_doc_freq': 1.0, 
#     'MAX_SENT_LEN': MAX_SENT_LEN,
#     'MAX_ORIG_TOKEN_LEN': MAX_ORIG_TOKEN_LEN,
    
#     #TokenPOSTagger
#     'emb_size': 64,
#     'res_block': wisResidualBlock,
#     'sng_backbone_kwargs': {
#         'layers_n': 3, 
#         'kernel_size': 5, 
#         'dropout': 0.3,
#         'conv_layer': MyConv1d,
#         'batchnorm': None },
#     'ctx_backbone_kwargs': {
#         'layers_n': 3, 
#         'kernel_size': 3, 
#         'dropout': 0.3,
#         'conv_layer': MyConv1d,
#         'batchnorm': None },

#     #train_eval_loop
#     'lr': 5e-3,
#     'epoch_n': 30,
#     'batch': 64,
#     'device': 'cuda',
#     'stop_pat': 5,
#     'lr_sched_pat': 2,
#     'lr_sched_fac': 0.5
# }

# histories[exp_name] = {
#     'hyps': hyps
# }

# sentence_level_model = SentenceLevelPOSTagger(len(char_vocab), len(label2id), 
#                                               embedding_size=hyps['emb_size'], res_block=hyps['res_block'],
#                                               single_backbone_kwargs=hyps['sng_backbone_kwargs'],
#                                               context_backbone_kwargs=hyps['ctx_backbone_kwargs'])
# print('Количество параметров', sum(np.product(t.shape) for t in sentence_level_model.parameters()))

# (histories[exp_name]['train_history'],
#  histories[exp_name]['best_model']) = train_eval_loop(sentence_level_model,
#                                               train_dataset,
#                                               test_dataset,
#                                               F.cross_entropy,
#                                               lr=hyps['lr'],
#                                               epoch_n=hyps['epoch_n'],
#                                               batch_size=hyps['batch'],
#                                               device=hyps['device'],
#                                               early_stopping_patience=hyps['stop_pat'],
#                                               max_batches_per_epoch_train=500,
#                                               max_batches_per_epoch_val=300,
#                                               best_acc_type = 'acc',
#                                               experiment_name = exp_name,
#                                               lr_scheduler_ctor=lambda optim: torch.optim.lr_scheduler.ReduceLROnPlateau(optim, patience=hyps['lr_sched_pat'],
#                                                                                                                          factor=hyps['lr_sched_fac'],
#                                                                                                                          verbose=True))

In [ ]:
# #hyperparameters
# exp_name = 'ctx_kernel_size=5'
# hyps = {
#     'dataset': 'base',
#     'char_min_count': 5,
#     'char_max_doc_freq': 1.0, 
#     'MAX_SENT_LEN': MAX_SENT_LEN,
#     'MAX_ORIG_TOKEN_LEN': MAX_ORIG_TOKEN_LEN,
    
#     #TokenPOSTagger
#     'emb_size': 64,
#     'res_block': wisResidualBlock,
#     'sng_backbone_kwargs': {
#         'layers_n': 3, 
#         'kernel_size': 3, 
#         'dropout': 0.3,
#         'conv_layer': MyConv1d,
#         'batchnorm': None },
#     'ctx_backbone_kwargs': {
#         'layers_n': 3, 
#         'kernel_size': 5, 
#         'dropout': 0.3,
#         'conv_layer': MyConv1d,
#         'batchnorm': None },

#     #train_eval_loop
#     'lr': 5e-3,
#     'epoch_n': 30,
#     'batch': 64,
#     'device': 'cuda',
#     'stop_pat': 5,
#     'lr_sched_pat': 2,
#     'lr_sched_fac': 0.5
# }

# histories[exp_name] = {
#     'hyps': hyps
# }

# sentence_level_model = SentenceLevelPOSTagger(len(char_vocab), len(label2id), 
#                                               embedding_size=hyps['emb_size'], res_block=hyps['res_block'],
#                                               single_backbone_kwargs=hyps['sng_backbone_kwargs'],
#                                               context_backbone_kwargs=hyps['ctx_backbone_kwargs'])
# print('Количество параметров', sum(np.product(t.shape) for t in sentence_level_model.parameters()))

# (histories[exp_name]['train_history'],
#  histories[exp_name]['best_model']) = train_eval_loop(sentence_level_model,
#                                               train_dataset,
#                                               test_dataset,
#                                               F.cross_entropy,
#                                               lr=hyps['lr'],
#                                               epoch_n=hyps['epoch_n'],
#                                               batch_size=hyps['batch'],
#                                               device=hyps['device'],
#                                               early_stopping_patience=hyps['stop_pat'],
#                                               max_batches_per_epoch_train=500,
#                                               max_batches_per_epoch_val=300,
#                                               best_acc_type = 'acc',
#                                               experiment_name = exp_name,
#                                               lr_scheduler_ctor=lambda optim: torch.optim.lr_scheduler.ReduceLROnPlateau(optim, patience=hyps['lr_sched_pat'],
#                                                                                                                          factor=hyps['lr_sched_fac'],
#                                                                                                                          verbose=True))

In [ ]:
# #hyperparameters
# exp_name = 'kernel_size=5'
# hyps = {
#     'dataset': 'base',
#     'char_min_count': 5,
#     'char_max_doc_freq': 1.0, 
#     'MAX_SENT_LEN': MAX_SENT_LEN,
#     'MAX_ORIG_TOKEN_LEN': MAX_ORIG_TOKEN_LEN,
    
#     #TokenPOSTagger
#     'emb_size': 64,
#     'res_block': wisResidualBlock,
#     'sng_backbone_kwargs': {
#         'layers_n': 3, 
#         'kernel_size': 5, 
#         'dropout': 0.3,
#         'conv_layer': MyConv1d,
#         'batchnorm': None },
#     'ctx_backbone_kwargs': {
#         'layers_n': 3, 
#         'kernel_size': 5, 
#         'dropout': 0.3,
#         'conv_layer': MyConv1d,
#         'batchnorm': None },

#     #train_eval_loop
#     'lr': 5e-3,
#     'epoch_n': 30,
#     'batch': 64,
#     'device': 'cuda',
#     'stop_pat': 5,
#     'lr_sched_pat': 2,
#     'lr_sched_fac': 0.5
# }

# histories[exp_name] = {
#     'hyps': hyps
# }

# sentence_level_model = SentenceLevelPOSTagger(len(char_vocab), len(label2id), 
#                                               embedding_size=hyps['emb_size'], res_block=hyps['res_block'],
#                                               single_backbone_kwargs=hyps['sng_backbone_kwargs'],
#                                               context_backbone_kwargs=hyps['ctx_backbone_kwargs'])
# print('Количество параметров', sum(np.product(t.shape) for t in sentence_level_model.parameters()))

# (histories[exp_name]['train_history'],
#  histories[exp_name]['best_model']) = train_eval_loop(sentence_level_model,
#                                               train_dataset,
#                                               test_dataset,
#                                               F.cross_entropy,
#                                               lr=hyps['lr'],
#                                               epoch_n=hyps['epoch_n'],
#                                               batch_size=hyps['batch'],
#                                               device=hyps['device'],
#                                               early_stopping_patience=hyps['stop_pat'],
#                                               max_batches_per_epoch_train=500,
#                                               max_batches_per_epoch_val=300,
#                                               best_acc_type = 'acc',
#                                               experiment_name = exp_name,
#                                               lr_scheduler_ctor=lambda optim: torch.optim.lr_scheduler.ReduceLROnPlateau(optim, patience=hyps['lr_sched_pat'],
#                                                                                                                          factor=hyps['lr_sched_fac'],
#                                                                                                                          verbose=True))

In [ ]:
# #hyperparameters
# exp_name = 'batch=128'
# hyps = {
#     'dataset': 'base',
#     'char_min_count': 5,
#     'char_max_doc_freq': 1.0, 
#     'MAX_SENT_LEN': MAX_SENT_LEN,
#     'MAX_ORIG_TOKEN_LEN': MAX_ORIG_TOKEN_LEN,
    
#     #TokenPOSTagger
#     'emb_size': 64,
#     'res_block': wisResidualBlock,
#     'sng_backbone_kwargs': {
#         'layers_n': 3, 
#         'kernel_size': 3, 
#         'dropout': 0.3,
#         'conv_layer': MyConv1d,
#         'batchnorm': None },
#     'ctx_backbone_kwargs': {
#         'layers_n': 3, 
#         'kernel_size': 3, 
#         'dropout': 0.3,
#         'conv_layer': MyConv1d,
#         'batchnorm': None },

#     #train_eval_loop
#     'lr': 5e-3,
#     'epoch_n': 30,
#     'batch': 128,
#     'device': 'cuda',
#     'stop_pat': 5,
#     'lr_sched_pat': 2,
#     'lr_sched_fac': 0.5
# }

# histories[exp_name] = {
#     'hyps': hyps
# }

# sentence_level_model = SentenceLevelPOSTagger(len(char_vocab), len(label2id), 
#                                               embedding_size=hyps['emb_size'], res_block=hyps['res_block'],
#                                               single_backbone_kwargs=hyps['sng_backbone_kwargs'],
#                                               context_backbone_kwargs=hyps['ctx_backbone_kwargs'])
# print('Количество параметров', sum(np.product(t.shape) for t in sentence_level_model.parameters()))

# (histories[exp_name]['train_history'],
#  histories[exp_name]['best_model']) = train_eval_loop(sentence_level_model,
#                                               train_dataset,
#                                               test_dataset,
#                                               F.cross_entropy,
#                                               lr=hyps['lr'],
#                                               epoch_n=hyps['epoch_n'],
#                                               batch_size=hyps['batch'],
#                                               device=hyps['device'],
#                                               early_stopping_patience=hyps['stop_pat'],
#                                               max_batches_per_epoch_train=500,
#                                               max_batches_per_epoch_val=300,
#                                               best_acc_type = 'acc',
#                                               experiment_name = exp_name,
#                                               lr_scheduler_ctor=lambda optim: torch.optim.lr_scheduler.ReduceLROnPlateau(optim, patience=hyps['lr_sched_pat'],
#                                                                                                                          factor=hyps['lr_sched_fac'],
#                                                                                                                          verbose=True))

In [ ]:
# #hyperparameters
# exp_name = 'batch=256'
# hyps = {
#     'dataset': 'base',
#     'char_min_count': 5,
#     'char_max_doc_freq': 1.0, 
#     'MAX_SENT_LEN': MAX_SENT_LEN,
#     'MAX_ORIG_TOKEN_LEN': MAX_ORIG_TOKEN_LEN,
    
#     #TokenPOSTagger
#     'emb_size': 64,
#     'res_block': wisResidualBlock,
#     'sng_backbone_kwargs': {
#         'layers_n': 3, 
#         'kernel_size': 3, 
#         'dropout': 0.3,
#         'conv_layer': MyConv1d,
#         'batchnorm': None },
#     'ctx_backbone_kwargs': {
#         'layers_n': 3, 
#         'kernel_size': 3, 
#         'dropout': 0.3,
#         'conv_layer': MyConv1d,
#         'batchnorm': None },

#     #train_eval_loop
#     'lr': 5e-3,
#     'epoch_n': 30,
#     'batch': 256,
#     'device': 'cuda',
#     'stop_pat': 5,
#     'lr_sched_pat': 2,
#     'lr_sched_fac': 0.5
# }

# histories[exp_name] = {
#     'hyps': hyps
# }

# sentence_level_model = SentenceLevelPOSTagger(len(char_vocab), len(label2id), 
#                                               embedding_size=hyps['emb_size'], res_block=hyps['res_block'],
#                                               single_backbone_kwargs=hyps['sng_backbone_kwargs'],
#                                               context_backbone_kwargs=hyps['ctx_backbone_kwargs'])
# print('Количество параметров', sum(np.product(t.shape) for t in sentence_level_model.parameters()))

# (histories[exp_name]['train_history'],
#  histories[exp_name]['best_model']) = train_eval_loop(sentence_level_model,
#                                               train_dataset,
#                                               test_dataset,
#                                               F.cross_entropy,
#                                               lr=hyps['lr'],
#                                               epoch_n=hyps['epoch_n'],
#                                               batch_size=hyps['batch'],
#                                               device=hyps['device'],
#                                               early_stopping_patience=hyps['stop_pat'],
#                                               max_batches_per_epoch_train=500,
#                                               max_batches_per_epoch_val=300,
#                                               best_acc_type = 'acc',
#                                               experiment_name = exp_name,
#                                               lr_scheduler_ctor=lambda optim: torch.optim.lr_scheduler.ReduceLROnPlateau(optim, patience=hyps['lr_sched_pat'],
#                                                                                                                          factor=hyps['lr_sched_fac'],
#                                                                                                                          verbose=True))

In [ ]:
# #hyperparameters
# exp_name = 'sng_layers_n=6'
# hyps = {
#     'dataset': 'base',
#     'char_min_count': 5,
#     'char_max_doc_freq': 1.0, 
#     'MAX_SENT_LEN': MAX_SENT_LEN,
#     'MAX_ORIG_TOKEN_LEN': MAX_ORIG_TOKEN_LEN,
    
#     #TokenPOSTagger
#     'emb_size': 64,
#     'res_block': wisResidualBlock,
#     'sng_backbone_kwargs': {
#         'layers_n': 6, 
#         'kernel_size': 3, 
#         'dropout': 0.3,
#         'conv_layer': MyConv1d,
#         'batchnorm': None },
#     'ctx_backbone_kwargs': {
#         'layers_n': 3, 
#         'kernel_size': 3, 
#         'dropout': 0.3,
#         'conv_layer': MyConv1d,
#         'batchnorm': None },

#     #train_eval_loop
#     'lr': 5e-3,
#     'epoch_n': 30,
#     'batch': 64,
#     'device': 'cuda',
#     'stop_pat': 5,
#     'lr_sched_pat': 2,
#     'lr_sched_fac': 0.5
# }

# histories[exp_name] = {
#     'hyps': hyps
# }

# sentence_level_model = SentenceLevelPOSTagger(len(char_vocab), len(label2id), 
#                                               embedding_size=hyps['emb_size'], res_block=hyps['res_block'],
#                                               single_backbone_kwargs=hyps['sng_backbone_kwargs'],
#                                               context_backbone_kwargs=hyps['ctx_backbone_kwargs'])
# print('Количество параметров', sum(np.product(t.shape) for t in sentence_level_model.parameters()))

# (histories[exp_name]['train_history'],
#  histories[exp_name]['best_model']) = train_eval_loop(sentence_level_model,
#                                               train_dataset,
#                                               test_dataset,
#                                               F.cross_entropy,
#                                               lr=hyps['lr'],
#                                               epoch_n=hyps['epoch_n'],
#                                               batch_size=hyps['batch'],
#                                               device=hyps['device'],
#                                               early_stopping_patience=hyps['stop_pat'],
#                                               max_batches_per_epoch_train=500,
#                                               max_batches_per_epoch_val=300,
#                                               best_acc_type = 'acc',
#                                               experiment_name = exp_name,
#                                               lr_scheduler_ctor=lambda optim: torch.optim.lr_scheduler.ReduceLROnPlateau(optim, patience=hyps['lr_sched_pat'],
#                                                                                                                          factor=hyps['lr_sched_fac'],
#                                                                                                                          verbose=True))

In [ ]:
# #hyperparameters
# exp_name = 'sng_layers_n=12'
# hyps = {
#     'dataset': 'base',
#     'char_min_count': 5,
#     'char_max_doc_freq': 1.0, 
#     'MAX_SENT_LEN': MAX_SENT_LEN,
#     'MAX_ORIG_TOKEN_LEN': MAX_ORIG_TOKEN_LEN,
    
#     #TokenPOSTagger
#     'emb_size': 64,
#     'res_block': wisResidualBlock,
#     'sng_backbone_kwargs': {
#         'layers_n': 16, 
#         'kernel_size': 3, 
#         'dropout': 0.3,
#         'conv_layer': MyConv1d,
#         'batchnorm': None },
#     'ctx_backbone_kwargs': {
#         'layers_n': 3, 
#         'kernel_size': 3, 
#         'dropout': 0.3,
#         'conv_layer': MyConv1d,
#         'batchnorm': None },

#     #train_eval_loop
#     'lr': 5e-3,
#     'epoch_n': 30,
#     'batch': 64,
#     'device': 'cuda',
#     'stop_pat': 5,
#     'lr_sched_pat': 2,
#     'lr_sched_fac': 0.5
# }

# histories[exp_name] = {
#     'hyps': hyps
# }

# sentence_level_model = SentenceLevelPOSTagger(len(char_vocab), len(label2id), 
#                                               embedding_size=hyps['emb_size'], res_block=hyps['res_block'],
#                                               single_backbone_kwargs=hyps['sng_backbone_kwargs'],
#                                               context_backbone_kwargs=hyps['ctx_backbone_kwargs'])
# print('Количество параметров', sum(np.product(t.shape) for t in sentence_level_model.parameters()))

# (histories[exp_name]['train_history'],
#  histories[exp_name]['best_model']) = train_eval_loop(sentence_level_model,
#                                               train_dataset,
#                                               test_dataset,
#                                               F.cross_entropy,
#                                               lr=hyps['lr'],
#                                               epoch_n=hyps['epoch_n'],
#                                               batch_size=hyps['batch'],
#                                               device=hyps['device'],
#                                               early_stopping_patience=hyps['stop_pat'],
#                                               max_batches_per_epoch_train=500,
#                                               max_batches_per_epoch_val=300,
#                                               best_acc_type = 'acc',
#                                               experiment_name = exp_name,
#                                               lr_scheduler_ctor=lambda optim: torch.optim.lr_scheduler.ReduceLROnPlateau(optim, patience=hyps['lr_sched_pat'],
#                                                                                                                          factor=hyps['lr_sched_fac'],
#                                                                                                                          verbose=True))

In [ ]:
# #hyperparameters
# exp_name = 'ctx_layers_n=6'
# hyps = {
#     'dataset': 'base',
#     'char_min_count': 5,
#     'char_max_doc_freq': 1.0, 
#     'MAX_SENT_LEN': MAX_SENT_LEN,
#     'MAX_ORIG_TOKEN_LEN': MAX_ORIG_TOKEN_LEN,
    
#     #TokenPOSTagger
#     'emb_size': 64,
#     'res_block': wisResidualBlock,
#     'sng_backbone_kwargs': {
#         'layers_n': 3, 
#         'kernel_size': 3, 
#         'dropout': 0.3,
#         'conv_layer': MyConv1d,
#         'batchnorm': None },
#     'ctx_backbone_kwargs': {
#         'layers_n': 6, 
#         'kernel_size': 3, 
#         'dropout': 0.3,
#         'conv_layer': MyConv1d,
#         'batchnorm': None },

#     #train_eval_loop
#     'lr': 5e-3,
#     'epoch_n': 30,
#     'batch': 64,
#     'device': 'cuda',
#     'stop_pat': 5,
#     'lr_sched_pat': 2,
#     'lr_sched_fac': 0.5
# }

# histories[exp_name] = {
#     'hyps': hyps
# }

# sentence_level_model = SentenceLevelPOSTagger(len(char_vocab), len(label2id), 
#                                               embedding_size=hyps['emb_size'], res_block=hyps['res_block'],
#                                               single_backbone_kwargs=hyps['sng_backbone_kwargs'],
#                                               context_backbone_kwargs=hyps['ctx_backbone_kwargs'])
# print('Количество параметров', sum(np.product(t.shape) for t in sentence_level_model.parameters()))

# (histories[exp_name]['train_history'],
#  histories[exp_name]['best_model']) = train_eval_loop(sentence_level_model,
#                                               train_dataset,
#                                               test_dataset,
#                                               F.cross_entropy,
#                                               lr=hyps['lr'],
#                                               epoch_n=hyps['epoch_n'],
#                                               batch_size=hyps['batch'],
#                                               device=hyps['device'],
#                                               early_stopping_patience=hyps['stop_pat'],
#                                               max_batches_per_epoch_train=500,
#                                               max_batches_per_epoch_val=300,
#                                               best_acc_type = 'acc',
#                                               experiment_name = exp_name,
#                                               lr_scheduler_ctor=lambda optim: torch.optim.lr_scheduler.ReduceLROnPlateau(optim, patience=hyps['lr_sched_pat'],
#                                                                                                                          factor=hyps['lr_sched_fac'],
#                                                                                                                          verbose=True))

In [ ]:
# #hyperparameters
# exp_name = 'ctx_layers_n=12'
# hyps = {
#     'dataset': 'base',
#     'char_min_count': 5,
#     'char_max_doc_freq': 1.0, 
#     'MAX_SENT_LEN': MAX_SENT_LEN,
#     'MAX_ORIG_TOKEN_LEN': MAX_ORIG_TOKEN_LEN,
    
#     #TokenPOSTagger
#     'emb_size': 64,
#     'res_block': wisResidualBlock,
#     'sng_backbone_kwargs': {
#         'layers_n': 3, 
#         'kernel_size': 3, 
#         'dropout': 0.3,
#         'conv_layer': MyConv1d,
#         'batchnorm': None },
#     'ctx_backbone_kwargs': {
#         'layers_n': 12, 
#         'kernel_size': 3, 
#         'dropout': 0.3,
#         'conv_layer': MyConv1d,
#         'batchnorm': None },

#     #train_eval_loop
#     'lr': 5e-3,
#     'epoch_n': 30,
#     'batch': 64,
#     'device': 'cuda',
#     'stop_pat': 5,
#     'lr_sched_pat': 2,
#     'lr_sched_fac': 0.5
# }

# histories[exp_name] = {
#     'hyps': hyps
# }

# sentence_level_model = SentenceLevelPOSTagger(len(char_vocab), len(label2id), 
#                                               embedding_size=hyps['emb_size'], res_block=hyps['res_block'],
#                                               single_backbone_kwargs=hyps['sng_backbone_kwargs'],
#                                               context_backbone_kwargs=hyps['ctx_backbone_kwargs'])
# print('Количество параметров', sum(np.product(t.shape) for t in sentence_level_model.parameters()))

# (histories[exp_name]['train_history'],
#  histories[exp_name]['best_model']) = train_eval_loop(sentence_level_model,
#                                               train_dataset,
#                                               test_dataset,
#                                               F.cross_entropy,
#                                               lr=hyps['lr'],
#                                               epoch_n=hyps['epoch_n'],
#                                               batch_size=hyps['batch'],
#                                               device=hyps['device'],
#                                               early_stopping_patience=hyps['stop_pat'],
#                                               max_batches_per_epoch_train=500,
#                                               max_batches_per_epoch_val=300,
#                                               best_acc_type = 'acc',
#                                               experiment_name = exp_name,
#                                               lr_scheduler_ctor=lambda optim: torch.optim.lr_scheduler.ReduceLROnPlateau(optim, patience=hyps['lr_sched_pat'],
#                                                                                                                          factor=hyps['lr_sched_fac'],
#                                                                                                                          verbose=True))

In [ ]:
# #hyperparameters
# exp_name = 'layers_n=6'
# hyps = {
#     'dataset': 'base',
#     'char_min_count': 5,
#     'char_max_doc_freq': 1.0, 
#     'MAX_SENT_LEN': MAX_SENT_LEN,
#     'MAX_ORIG_TOKEN_LEN': MAX_ORIG_TOKEN_LEN,
    
#     #TokenPOSTagger
#     'emb_size': 64,
#     'res_block': wisResidualBlock,
#     'sng_backbone_kwargs': {
#         'layers_n': 6,
#         'kernel_size': 3,
#         'dropout': 0.3,
#         'conv_layer': MyConv1d,
#         'batchnorm': None },
#     'ctx_backbone_kwargs': {
#         'layers_n': 6,
#         'kernel_size': 3,
#         'dropout': 0.3,
#         'conv_layer': MyConv1d,
#         'batchnorm': None },

#     #train_eval_loop
#     'lr': 5e-3,
#     'epoch_n': 30,
#     'batch': 64,
#     'device': 'cuda',
#     'stop_pat': 5,
#     'lr_sched_pat': 2,
#     'lr_sched_fac': 0.5
# }

# histories[exp_name] = {
#     'hyps': hyps
# }

# sentence_level_model = SentenceLevelPOSTagger(len(char_vocab), len(label2id), 
#                                               embedding_size=hyps['emb_size'], res_block=hyps['res_block'],
#                                               single_backbone_kwargs=hyps['sng_backbone_kwargs'],
#                                               context_backbone_kwargs=hyps['ctx_backbone_kwargs'])
# print('Количество параметров', sum(np.product(t.shape) for t in sentence_level_model.parameters()))

# (histories[exp_name]['train_history'],
#  histories[exp_name]['best_model']) = train_eval_loop(sentence_level_model,
#                                               train_dataset,
#                                               test_dataset,
#                                               F.cross_entropy,
#                                               lr=hyps['lr'],
#                                               epoch_n=hyps['epoch_n'],
#                                               batch_size=hyps['batch'],
#                                               device=hyps['device'],
#                                               early_stopping_patience=hyps['stop_pat'],
#                                               max_batches_per_epoch_train=500,
#                                               max_batches_per_epoch_val=300,
#                                               best_acc_type = 'acc',
#                                               experiment_name = exp_name,
#                                               lr_scheduler_ctor=lambda optim: torch.optim.lr_scheduler.ReduceLROnPlateau(optim, patience=hyps['lr_sched_pat'],
#                                                                                                                          factor=hyps['lr_sched_fac'],
#                                                                                                                          verbose=True))

In [ ]:
# #hyperparameters
# exp_name = 'layers_n=12'
# hyps = {
#     'dataset': 'base',
#     'char_min_count': 5,
#     'char_max_doc_freq': 1.0, 
#     'MAX_SENT_LEN': MAX_SENT_LEN,
#     'MAX_ORIG_TOKEN_LEN': MAX_ORIG_TOKEN_LEN,
    
#     #TokenPOSTagger
#     'emb_size': 64,
#     'res_block': wisResidualBlock,
#     'sng_backbone_kwargs': {
#         'layers_n': 12,
#         'kernel_size': 3,
#         'dropout': 0.3,
#         'conv_layer': MyConv1d,
#         'batchnorm': None },
#     'ctx_backbone_kwargs': {
#         'layers_n': 12,
#         'kernel_size': 3,
#         'dropout': 0.3,
#         'conv_layer': MyConv1d,
#         'batchnorm': None },

#     #train_eval_loop
#     'lr': 5e-3,
#     'epoch_n': 30,
#     'batch': 64,
#     'device': 'cuda',
#     'stop_pat': 5,
#     'lr_sched_pat': 2,
#     'lr_sched_fac': 0.5
# }

# histories[exp_name] = {
#     'hyps': hyps
# }

# sentence_level_model = SentenceLevelPOSTagger(len(char_vocab), len(label2id), 
#                                               embedding_size=hyps['emb_size'], res_block=hyps['res_block'],
#                                               single_backbone_kwargs=hyps['sng_backbone_kwargs'],
#                                               context_backbone_kwargs=hyps['ctx_backbone_kwargs'])
# print('Количество параметров', sum(np.product(t.shape) for t in sentence_level_model.parameters()))

# (histories[exp_name]['train_history'],
#  histories[exp_name]['best_model']) = train_eval_loop(sentence_level_model,
#                                               train_dataset,
#                                               test_dataset,
#                                               F.cross_entropy,
#                                               lr=hyps['lr'],
#                                               epoch_n=hyps['epoch_n'],
#                                               batch_size=hyps['batch'],
#                                               device=hyps['device'],
#                                               early_stopping_patience=hyps['stop_pat'],
#                                               max_batches_per_epoch_train=500,
#                                               max_batches_per_epoch_val=300,
#                                               best_acc_type = 'acc',
#                                               experiment_name = exp_name,
#                                               lr_scheduler_ctor=lambda optim: torch.optim.lr_scheduler.ReduceLROnPlateau(optim, patience=hyps['lr_sched_pat'],
#                                                                                                                          factor=hyps['lr_sched_fac'],
#                                                                                                                          verbose=True))

### Эксперимент - подключить прореженные (dilated) свёртки, чтобы увеличить рецептивное поле без увеличения числа параметров

In [ ]:
class wisResidualBlock(torch.nn.Module):
    #v1.1 with dilation
    def __init__(self, features_num, kernel_size=3, conv_layer=nn.Conv1d, dropout=0.0, batchnorm = None, dilation=1):
        #batchnorm = None, nn.InstanceNorm1d, nn.BatchNorm1d
        
        super().__init__()
        if batchnorm is None:
            self.layer = nn.Sequential(
                conv_layer(features_num, features_num, kernel_size, padding=(kernel_size//2)*dilation, dilation=dilation),
                nn.Dropout(dropout),
                nn.LeakyReLU())
        else:
            self.layer = nn.Sequential(
                conv_layer(features_num, features_num, kernel_size, padding=(kernel_size//2)*dilation, dilation=dilation),
                batchnorm(features_num),
                nn.Dropout(dropout),
                nn.LeakyReLU())
            
    def forward(self, x):        
        return x + self.layer(x)

In [ ]:
class wisStackedConv1d(nn.Module):
    #v1.1 with dilation
    def __init__(self, features_num, layers_n=1, res_block = wisResidualBlock, 
                 kernel_size=3, conv_layer=nn.Conv1d, dropout=0.0, batchnorm = None, dilation=False):
        super().__init__()
        if not dilation:
            self.layers = nn.Sequential(*[res_block(features_num, 
                                                    kernel_size=kernel_size, 
                                                    conv_layer=conv_layer,
                                                    dropout=dropout,
                                                    batchnorm=batchnorm) for _ in range(layers_n)])
        else:
            self.layers = nn.Sequential(*[res_block(features_num, 
                                                    kernel_size=kernel_size, 
                                                    conv_layer=conv_layer,
                                                    dropout=dropout,
                                                    batchnorm=batchnorm,
                                                    dilation=2**i) for i in range(layers_n)])
    
    def forward(self, x):
        """x - BatchSize x FeaturesNum x SequenceLen"""
        return self.layers(x)

In [ ]:
# #hyperparameters
# exp_name = 'ctx_deliated'
# hyps = {
#     'dataset': 'base',
#     'char_min_count': 5,
#     'char_max_doc_freq': 1.0, 
#     'MAX_SENT_LEN': MAX_SENT_LEN,
#     'MAX_ORIG_TOKEN_LEN': MAX_ORIG_TOKEN_LEN,
    
#     #TokenPOSTagger
#     'emb_size': 64,
#     'res_block': wisResidualBlock,
#     'sng_backbone_kwargs': {
#         'layers_n': 3, 
#         'kernel_size': 3, 
#         'dropout': 0.3,
#         'conv_layer': nn.Conv1d,
#         'batchnorm': None,
#         'dilation': False },
#     'ctx_backbone_kwargs': {
#         'layers_n': 3,
#         'kernel_size': 3,
#         'dropout': 0.3,
#         'conv_layer': nn.Conv1d,
#         'batchnorm': None,
#         'dilation': True  },

#     #train_eval_loop
#     'lr': 5e-3,
#     'epoch_n': 30,
#     'batch': 64,
#     'device': 'cuda',
#     'stop_pat': 5,
#     'lr_sched_pat': 2,
#     'lr_sched_fac': 0.5
# }

# histories[exp_name] = {
#     'hyps': hyps
# }

# #Dilated conv is too slow - https://github.com/pytorch/pytorch/issues/15054
# torch.backends.cudnn.benchmark=True
# torch.backends.cudnn.deterministic=False

# sentence_level_model = SentenceLevelPOSTagger(len(char_vocab), len(label2id), 
#                                               embedding_size=hyps['emb_size'], res_block=hyps['res_block'],
#                                               single_backbone_kwargs=hyps['sng_backbone_kwargs'],
#                                               context_backbone_kwargs=hyps['ctx_backbone_kwargs'])
# print('Количество параметров', sum(np.product(t.shape) for t in sentence_level_model.parameters()))

# (histories[exp_name]['train_history'],
#  histories[exp_name]['best_model']) = train_eval_loop(sentence_level_model,
#                                               train_dataset,
#                                               test_dataset,
#                                               F.cross_entropy,
#                                               lr=hyps['lr'],
#                                               epoch_n=hyps['epoch_n'],
#                                               batch_size=hyps['batch'],
#                                               device=hyps['device'],
#                                               early_stopping_patience=hyps['stop_pat'],
#                                               max_batches_per_epoch_train=500,
#                                               max_batches_per_epoch_val=300,
#                                               best_acc_type = 'acc',
#                                               experiment_name = exp_name,
#                                               lr_scheduler_ctor=lambda optim: torch.optim.lr_scheduler.ReduceLROnPlateau(optim, patience=hyps['lr_sched_pat'],
#                                                                                                                          factor=hyps['lr_sched_fac'],
#                                                                                                                          verbose=True))

In [ ]:
# #hyperparameters
# exp_name = 'ctx_deliated_ctx_layers_n=6'
# hyps = {
#     'dataset': 'base',
#     'char_min_count': 5,
#     'char_max_doc_freq': 1.0, 
#     'MAX_SENT_LEN': MAX_SENT_LEN,
#     'MAX_ORIG_TOKEN_LEN': MAX_ORIG_TOKEN_LEN,
    
#     #TokenPOSTagger
#     'emb_size': 64,
#     'res_block': wisResidualBlock,
#     'sng_backbone_kwargs': {
#         'layers_n': 3, 
#         'kernel_size': 3, 
#         'dropout': 0.3,
#         'conv_layer': nn.Conv1d,
#         'batchnorm': None,
#         'dilation': False },
#     'ctx_backbone_kwargs': {
#         'layers_n': 6,
#         'kernel_size': 3,
#         'dropout': 0.3,
#         'conv_layer': nn.Conv1d,
#         'batchnorm': None,
#         'dilation': True  },

#     #train_eval_loop
#     'lr': 5e-3,
#     'epoch_n': 30,
#     'batch': 64,
#     'device': 'cuda',
#     'stop_pat': 5,
#     'lr_sched_pat': 2,
#     'lr_sched_fac': 0.5
# }

# histories[exp_name] = {
#     'hyps': hyps
# }

# #Dilated conv is too slow - https://github.com/pytorch/pytorch/issues/15054
# torch.backends.cudnn.benchmark=True
# torch.backends.cudnn.deterministic=False

# sentence_level_model = SentenceLevelPOSTagger(len(char_vocab), len(label2id), 
#                                               embedding_size=hyps['emb_size'], res_block=hyps['res_block'],
#                                               single_backbone_kwargs=hyps['sng_backbone_kwargs'],
#                                               context_backbone_kwargs=hyps['ctx_backbone_kwargs'])
# print('Количество параметров', sum(np.product(t.shape) for t in sentence_level_model.parameters()))

# (histories[exp_name]['train_history'],
#  histories[exp_name]['best_model']) = train_eval_loop(sentence_level_model,
#                                               train_dataset,
#                                               test_dataset,
#                                               F.cross_entropy,
#                                               lr=hyps['lr'],
#                                               epoch_n=hyps['epoch_n'],
#                                               batch_size=hyps['batch'],
#                                               device=hyps['device'],
#                                               early_stopping_patience=hyps['stop_pat'],
#                                               max_batches_per_epoch_train=500,
#                                               max_batches_per_epoch_val=300,
#                                               best_acc_type = 'acc',
#                                               experiment_name = exp_name,
#                                               lr_scheduler_ctor=lambda optim: torch.optim.lr_scheduler.ReduceLROnPlateau(optim, patience=hyps['lr_sched_pat'],
#                                                                                                                          factor=hyps['lr_sched_fac'],
#                                                                                                                          verbose=True))

In [ ]:
# #hyperparameters
# exp_name = 'sng_deliated'
# hyps = {
#     'dataset': 'base',
#     'char_min_count': 5,
#     'char_max_doc_freq': 1.0, 
#     'MAX_SENT_LEN': MAX_SENT_LEN,
#     'MAX_ORIG_TOKEN_LEN': MAX_ORIG_TOKEN_LEN,
    
#     #TokenPOSTagger
#     'emb_size': 64,
#     'res_block': wisResidualBlock,
#     'sng_backbone_kwargs': {
#         'layers_n': 3, 
#         'kernel_size': 3, 
#         'dropout': 0.3,
#         'conv_layer': nn.Conv1d,
#         'batchnorm': None,
#         'dilation': True },
#     'ctx_backbone_kwargs': {
#         'layers_n': 3,
#         'kernel_size': 3,
#         'dropout': 0.3,
#         'conv_layer': nn.Conv1d,
#         'batchnorm': None,
#         'dilation': False  },

#     #train_eval_loop
#     'lr': 5e-3,
#     'epoch_n': 30,
#     'batch': 64,
#     'device': 'cuda',
#     'stop_pat': 5,
#     'lr_sched_pat': 2,
#     'lr_sched_fac': 0.5
# }

# histories[exp_name] = {
#     'hyps': hyps
# }

# #Dilated conv is too slow - https://github.com/pytorch/pytorch/issues/15054
# torch.backends.cudnn.benchmark=True
# torch.backends.cudnn.deterministic=False

# sentence_level_model = SentenceLevelPOSTagger(len(char_vocab), len(label2id), 
#                                               embedding_size=hyps['emb_size'], res_block=hyps['res_block'],
#                                               single_backbone_kwargs=hyps['sng_backbone_kwargs'],
#                                               context_backbone_kwargs=hyps['ctx_backbone_kwargs'])
# print('Количество параметров', sum(np.product(t.shape) for t in sentence_level_model.parameters()))

# (histories[exp_name]['train_history'],
#  histories[exp_name]['best_model']) = train_eval_loop(sentence_level_model,
#                                               train_dataset,
#                                               test_dataset,
#                                               F.cross_entropy,
#                                               lr=hyps['lr'],
#                                               epoch_n=hyps['epoch_n'],
#                                               batch_size=hyps['batch'],
#                                               device=hyps['device'],
#                                               early_stopping_patience=hyps['stop_pat'],
#                                               max_batches_per_epoch_train=500,
#                                               max_batches_per_epoch_val=300,
#                                               best_acc_type = 'acc',
#                                               experiment_name = exp_name,
#                                               lr_scheduler_ctor=lambda optim: torch.optim.lr_scheduler.ReduceLROnPlateau(optim, patience=hyps['lr_sched_pat'],
#                                                                                                                          factor=hyps['lr_sched_fac'],
#                                                                                                                          verbose=True))

In [ ]:
# #hyperparameters
# exp_name = 'all_deliated'
# hyps = {
#     'dataset': 'base',
#     'char_min_count': 5,
#     'char_max_doc_freq': 1.0, 
#     'MAX_SENT_LEN': MAX_SENT_LEN,
#     'MAX_ORIG_TOKEN_LEN': MAX_ORIG_TOKEN_LEN,
    
#     #TokenPOSTagger
#     'emb_size': 64,
#     'res_block': wisResidualBlock,
#     'sng_backbone_kwargs': {
#         'layers_n': 3, 
#         'kernel_size': 3, 
#         'dropout': 0.3,
#         'conv_layer': nn.Conv1d,
#         'batchnorm': None,
#         'dilation': True },
#     'ctx_backbone_kwargs': {
#         'layers_n': 3,
#         'kernel_size': 3,
#         'dropout': 0.3,
#         'conv_layer': nn.Conv1d,
#         'batchnorm': None,
#         'dilation': True  },

#     #train_eval_loop
#     'lr': 5e-3,
#     'epoch_n': 30,
#     'batch': 64,
#     'device': 'cuda',
#     'stop_pat': 5,
#     'lr_sched_pat': 2,
#     'lr_sched_fac': 0.5
# }

# histories[exp_name] = {
#     'hyps': hyps
# }

# #Dilated conv is too slow - https://github.com/pytorch/pytorch/issues/15054
# torch.backends.cudnn.benchmark=True
# torch.backends.cudnn.deterministic=False

# sentence_level_model = SentenceLevelPOSTagger(len(char_vocab), len(label2id), 
#                                               embedding_size=hyps['emb_size'], res_block=hyps['res_block'],
#                                               single_backbone_kwargs=hyps['sng_backbone_kwargs'],
#                                               context_backbone_kwargs=hyps['ctx_backbone_kwargs'])
# print('Количество параметров', sum(np.product(t.shape) for t in sentence_level_model.parameters()))

# (histories[exp_name]['train_history'],
#  histories[exp_name]['best_model']) = train_eval_loop(sentence_level_model,
#                                               train_dataset,
#                                               test_dataset,
#                                               F.cross_entropy,
#                                               lr=hyps['lr'],
#                                               epoch_n=hyps['epoch_n'],
#                                               batch_size=hyps['batch'],
#                                               device=hyps['device'],
#                                               early_stopping_patience=hyps['stop_pat'],
#                                               max_batches_per_epoch_train=500,
#                                               max_batches_per_epoch_val=300,
#                                               best_acc_type = 'acc',
#                                               experiment_name = exp_name,
#                                               lr_scheduler_ctor=lambda optim: torch.optim.lr_scheduler.ReduceLROnPlateau(optim, patience=hyps['lr_sched_pat'],
#                                                                                                                          factor=hyps['lr_sched_fac'],
#                                                                                                                          verbose=True))

In [ ]:
# #hyperparameters
# exp_name = 'all_deliated_layers_n=6'
# hyps = {
#     'dataset': 'base',
#     'char_min_count': 5,
#     'char_max_doc_freq': 1.0, 
#     'MAX_SENT_LEN': MAX_SENT_LEN,
#     'MAX_ORIG_TOKEN_LEN': MAX_ORIG_TOKEN_LEN,
    
#     #TokenPOSTagger
#     'emb_size': 64,
#     'res_block': wisResidualBlock,
#     'sng_backbone_kwargs': {
#         'layers_n': 6, 
#         'kernel_size': 3, 
#         'dropout': 0.3,
#         'conv_layer': nn.Conv1d,
#         'batchnorm': None,
#         'dilation': True },
#     'ctx_backbone_kwargs': {
#         'layers_n': 6,
#         'kernel_size': 3,
#         'dropout': 0.3,
#         'conv_layer': nn.Conv1d,
#         'batchnorm': None,
#         'dilation': True  },

#     #train_eval_loop
#     'lr': 5e-3,
#     'epoch_n': 30,
#     'batch': 64,
#     'device': 'cuda',
#     'stop_pat': 5,
#     'lr_sched_pat': 2,
#     'lr_sched_fac': 0.5
# }

# histories[exp_name] = {
#     'hyps': hyps
# }

# #Dilated conv is too slow - https://github.com/pytorch/pytorch/issues/15054
# torch.backends.cudnn.benchmark=True
# torch.backends.cudnn.deterministic=False

# sentence_level_model = SentenceLevelPOSTagger(len(char_vocab), len(label2id), 
#                                               embedding_size=hyps['emb_size'], res_block=hyps['res_block'],
#                                               single_backbone_kwargs=hyps['sng_backbone_kwargs'],
#                                               context_backbone_kwargs=hyps['ctx_backbone_kwargs'])
# print('Количество параметров', sum(np.product(t.shape) for t in sentence_level_model.parameters()))

# (histories[exp_name]['train_history'],
#  histories[exp_name]['best_model']) = train_eval_loop(sentence_level_model,
#                                               train_dataset,
#                                               test_dataset,
#                                               F.cross_entropy,
#                                               lr=hyps['lr'],
#                                               epoch_n=hyps['epoch_n'],
#                                               batch_size=hyps['batch'],
#                                               device=hyps['device'],
#                                               early_stopping_patience=hyps['stop_pat'],
#                                               max_batches_per_epoch_train=500,
#                                               max_batches_per_epoch_val=300,
#                                               best_acc_type = 'acc',
#                                               experiment_name = exp_name,
#                                               lr_scheduler_ctor=lambda optim: torch.optim.lr_scheduler.ReduceLROnPlateau(optim, patience=hyps['lr_sched_pat'],
#                                                                                                                          factor=hyps['lr_sched_fac'],
#                                                                                                                          verbose=True))

### Эксперимент - использовать в качестве обозначения начала и конца слова не 0, а какой-нибудь другой токен (для 0 nn.Embedding всегда выдаёт нулевой вектор, а в этом случае для начала а конца слова будут учиться специальные вектора)

In [ ]:
# #hyperparameters
# exp_name = 'custom_start_end_tags_01'
# hyps = {
#     'dataset': 'setags',
#     'char_min_count': 5,
#     'char_max_doc_freq': 1.0, 
#     'MAX_SENT_LEN': MAX_SENT_LEN,
#     'MAX_ORIG_TOKEN_LEN': MAX_ORIG_TOKEN_LEN,
#     'start_end_tag': True,
    
#     #TokenPOSTagger
#     'emb_size': 64,
#     'res_block': wisResidualBlock,
#     'sng_backbone_kwargs': {
#         'layers_n': 3, 
#         'kernel_size': 3, 
#         'dropout': 0.3ф,
#         'conv_layer': MyConv1d,
#         'batchnorm': None,
#         'dilation': False },
#     'ctx_backbone_kwargs': {
#         'layers_n': 3,
#         'kernel_size': 3,
#         'dropout': 0.3,
#         'conv_layer': MyConv1d,
#         'batchnorm': None,
#         'dilation': False  },

#     #train_eval_loop
#     'loss_func': F.cross_entropy,
#     'lr': 5e-3,
#     'epoch_n': 30,
#     'batch': 64,
#     'device': 'cuda',
#     'stop_pat': 5,
#     'lr_sched_pat': 2,
#     'lr_sched_fac': 0.5
# }

# histories[exp_name] = {
#     'hyps': hyps
# }

# train_char_tokenized = tokenize_corpus(all_train_texts, tokenizer=character_tokenize)
# char_vocab, word_doc_freq = build_vocabulary(train_char_tokenized, 
#                                              max_doc_freq=hyps['char_max_doc_freq'], 
#                                              min_count=hyps['char_min_count'], 
#                                              pad_word='<PAD>', 
#                                              start_end_tag=hyps['start_end_tag'])

# print("Количество уникальных символов", len(char_vocab))
# print(list(char_vocab.items())[:10])


# UNIQUE_TAGS = ['<NOTAG>'] + sorted({token.upos for sent in full_train for token in sent if token.upos})
# label2id = {label: i for i, label in enumerate(UNIQUE_TAGS)}

# train_inputs, train_labels = pos_corpus_to_tensor(full_train, char_vocab, label2id, MAX_SENT_LEN, MAX_ORIG_TOKEN_LEN)
# train_dataset = TensorDataset(train_inputs, train_labels)

# test_inputs, test_labels = pos_corpus_to_tensor(full_test, char_vocab, label2id, MAX_SENT_LEN, MAX_ORIG_TOKEN_LEN)
# test_dataset = TensorDataset(test_inputs, test_labels)

# if hyps['start_end_tag']:
#     #for calc final statistics
#     hdatasets['setags'] = {}
#     hdatasets['setags']['char_vocab'] = char_vocab
#     hdatasets['setags']['train_labels'] = train_labels
#     hdatasets['setags']['train_dataset'] = train_dataset
#     hdatasets['setags']['test_labels'] = test_labels
#     hdatasets['setags']['test_dataset'] = test_dataset

# sentence_level_model = SentenceLevelPOSTagger(len(char_vocab), len(label2id), 
#                                               embedding_size=hyps['emb_size'], res_block=hyps['res_block'],
#                                               single_backbone_kwargs=hyps['sng_backbone_kwargs'],
#                                               context_backbone_kwargs=hyps['ctx_backbone_kwargs'])
# print('Количество параметров', sum(np.product(t.shape) for t in sentence_level_model.parameters()))

# (histories[exp_name]['train_history'],
#  histories[exp_name]['best_model']) = train_eval_loop(sentence_level_model,
#                                               train_dataset,
#                                               test_dataset,
#                                               hyps['loss_func'],
#                                               lr=hyps['lr'],
#                                               epoch_n=hyps['epoch_n'],
#                                               batch_size=hyps['batch'],
#                                               device=hyps['device'],
#                                               early_stopping_patience=hyps['stop_pat'],
#                                               max_batches_per_epoch_train=500,
#                                               max_batches_per_epoch_val=300,
#                                               best_acc_type = 'acc',
#                                               experiment_name = exp_name,
#                                               lr_scheduler_ctor=lambda optim: torch.optim.lr_scheduler.ReduceLROnPlateau(optim, patience=hyps['lr_sched_pat'],
#                                                                                                                          factor=hyps['lr_sched_fac'],
#                                                                                                                          verbose=True))

In [ ]:
# #hyperparameters
# exp_name = 'custom_start_end_tags_02'
# hyps = {
#     'dataset': 'setags',
#     'char_min_count': 5,
#     'char_max_doc_freq': 1.0, 
#     'MAX_SENT_LEN': MAX_SENT_LEN,
#     'MAX_ORIG_TOKEN_LEN': MAX_ORIG_TOKEN_LEN,
#     'start_end_tag': True,
    
#     #TokenPOSTagger
#     'emb_size': 64,
#     'res_block': wisResidualBlock,
#     'sng_backbone_kwargs': {
#         'layers_n': 3, 
#         'kernel_size': 3, 
#         'dropout': 0.3,
#         'conv_layer': MyConv1d,
#         'batchnorm': None,
#         'dilation': False },
#     'ctx_backbone_kwargs': {
#         'layers_n': 3,
#         'kernel_size': 3,
#         'dropout': 0.3,
#         'conv_layer': MyConv1d,
#         'batchnorm': None,
#         'dilation': False  },

#     #train_eval_loop
#     'loss_func': F.cross_entropy,
#     'lr': 5e-3,
#     'epoch_n': 30,
#     'batch': 64,
#     'device': 'cuda',
#     'stop_pat': 5,
#     'lr_sched_pat': 2,
#     'lr_sched_fac': 0.5
# }

# histories[exp_name] = {
#     'hyps': hyps
# }

# train_char_tokenized = tokenize_corpus(all_train_texts, tokenizer=character_tokenize)
# char_vocab, word_doc_freq = build_vocabulary(train_char_tokenized, 
#                                              max_doc_freq=hyps['char_max_doc_freq'], 
#                                              min_count=hyps['char_min_count'], 
#                                              pad_word='<PAD>', 
#                                              start_end_tag=hyps['start_end_tag'])

# print("Количество уникальных символов", len(char_vocab))
# print(list(char_vocab.items())[:10])


# UNIQUE_TAGS = ['<NOTAG>'] + sorted({token.upos for sent in full_train for token in sent if token.upos})
# label2id = {label: i for i, label in enumerate(UNIQUE_TAGS)}

# train_inputs, train_labels = pos_corpus_to_tensor(full_train, char_vocab, label2id, MAX_SENT_LEN, MAX_ORIG_TOKEN_LEN)
# train_dataset = TensorDataset(train_inputs, train_labels)

# test_inputs, test_labels = pos_corpus_to_tensor(full_test, char_vocab, label2id, MAX_SENT_LEN, MAX_ORIG_TOKEN_LEN)
# test_dataset = TensorDataset(test_inputs, test_labels)

# if hyps['start_end_tag']:
#     #for calc final statistics
#     hdatasets['setags'] = {}
#     hdatasets['setags']['char_vocab'] = char_vocab
#     hdatasets['setags']['train_labels'] = train_labels
#     hdatasets['setags']['train_dataset'] = train_dataset
#     hdatasets['setags']['test_labels'] = test_labels
#     hdatasets['setags']['test_dataset'] = test_dataset

# sentence_level_model = SentenceLevelPOSTagger(len(char_vocab), len(label2id), 
#                                               embedding_size=hyps['emb_size'], res_block=hyps['res_block'],
#                                               single_backbone_kwargs=hyps['sng_backbone_kwargs'],
#                                               context_backbone_kwargs=hyps['ctx_backbone_kwargs'])
# print('Количество параметров', sum(np.product(t.shape) for t in sentence_level_model.parameters()))

# (histories[exp_name]['train_history'],
#  histories[exp_name]['best_model']) = train_eval_loop(sentence_level_model,
#                                               train_dataset,
#                                               test_dataset,
#                                               hyps['loss_func'],
#                                               lr=hyps['lr'],
#                                               epoch_n=hyps['epoch_n'],
#                                               batch_size=hyps['batch'],
#                                               device=hyps['device'],
#                                               early_stopping_patience=hyps['stop_pat'],
#                                               max_batches_per_epoch_train=500,
#                                               max_batches_per_epoch_val=300,
#                                               best_acc_type = 'acc',
#                                               experiment_name = exp_name,
#                                               lr_scheduler_ctor=lambda optim: torch.optim.lr_scheduler.ReduceLROnPlateau(optim, patience=hyps['lr_sched_pat'],
#                                                                                                                          factor=hyps['lr_sched_fac'],
#                                                                                                                          verbose=True))

### Эксперимент - добавить взвешивание классов

In [ ]:
# from sklearn.utils import class_weight
# class_weights = class_weight.compute_class_weight('balanced',np.unique(train_labels.data),
#                                                 train_labels.data.detach().cpu().numpy().reshape(-1))
# class_weights = torch.tensor(class_weights, dtype=torch.float).cuda()
# class_weights

# class_weights_log = torch.log(class_weights+1)
# class_weights_log

In [ ]:
# #hyperparameters
# exp_name = 'class_weights_log_01'
# hyps = {
#     'dataset': 'base',
#     'char_min_count': 5,
#     'char_max_doc_freq': 1.0, 
#     'MAX_SENT_LEN': MAX_SENT_LEN,
#     'MAX_ORIG_TOKEN_LEN': MAX_ORIG_TOKEN_LEN,
#     'start_end_tag': False,
    
#     #TokenPOSTagger
#     'emb_size': 64,
#     'res_block': wisResidualBlock,
#     'sng_backbone_kwargs': {
#         'layers_n': 3, 
#         'kernel_size': 3, 
#         'dropout': 0.3,
#         'conv_layer': MyConv1d,
#         'batchnorm': None,
#         'dilation': False },
#     'ctx_backbone_kwargs': {
#         'layers_n': 3,
#         'kernel_size': 3,
#         'dropout': 0.3,
#         'conv_layer': MyConv1d,
#         'batchnorm': None,
#         'dilation': False  },

#     #train_eval_loop
#     'loss_func': nn.CrossEntropyLoss(weight=class_weights_log,reduction='mean'),
#     'lr': 5e-3,
#     'epoch_n': 30,
#     'batch': 64,
#     'device': 'cuda',
#     'stop_pat': 5,
#     'lr_sched_pat': 2,
#     'lr_sched_fac': 0.5
# }

# histories[exp_name] = {
#     'hyps': hyps
# }

# train_char_tokenized = tokenize_corpus(all_train_texts, tokenizer=character_tokenize)
# char_vocab, word_doc_freq = build_vocabulary(train_char_tokenized, 
#                                              max_doc_freq=hyps['char_max_doc_freq'], 
#                                              min_count=hyps['char_min_count'], 
#                                              pad_word='<PAD>', 
#                                              start_end_tag=hyps['start_end_tag'])

# print("Количество уникальных символов", len(char_vocab))
# print(list(char_vocab.items())[:10])


# UNIQUE_TAGS = ['<NOTAG>'] + sorted({token.upos for sent in full_train for token in sent if token.upos})
# label2id = {label: i for i, label in enumerate(UNIQUE_TAGS)}

# train_inputs, train_labels = pos_corpus_to_tensor(full_train, char_vocab, label2id, MAX_SENT_LEN, MAX_ORIG_TOKEN_LEN)
# train_dataset = TensorDataset(train_inputs, train_labels)

# test_inputs, test_labels = pos_corpus_to_tensor(full_test, char_vocab, label2id, MAX_SENT_LEN, MAX_ORIG_TOKEN_LEN)
# test_dataset = TensorDataset(test_inputs, test_labels)

# if hyps['start_end_tag']:
#     #for calc final statistics
#     hdatasets['setags'] = {}
#     hdatasets['setags']['char_vocab'] = char_vocab
#     hdatasets['setags']['train_labels'] = train_labels
#     hdatasets['setags']['train_dataset'] = train_dataset
#     hdatasets['setags']['test_labels'] = test_labels
#     hdatasets['setags']['test_dataset'] = test_dataset

# sentence_level_model = SentenceLevelPOSTagger(len(char_vocab), len(label2id), 
#                                               embedding_size=hyps['emb_size'], res_block=hyps['res_block'],
#                                               single_backbone_kwargs=hyps['sng_backbone_kwargs'],
#                                               context_backbone_kwargs=hyps['ctx_backbone_kwargs'])
# print('Количество параметров', sum(np.product(t.shape) for t in sentence_level_model.parameters()))

# (histories[exp_name]['train_history'],
#  histories[exp_name]['best_model']) = train_eval_loop(sentence_level_model,
#                                               train_dataset,
#                                               test_dataset,
#                                               hyps['loss_func'],
#                                               lr=hyps['lr'],
#                                               epoch_n=hyps['epoch_n'],
#                                               batch_size=hyps['batch'],
#                                               device=hyps['device'],
#                                               early_stopping_patience=hyps['stop_pat'],
#                                               max_batches_per_epoch_train=500,
#                                               max_batches_per_epoch_val=300,
#                                               best_acc_type = 'acc',
#                                               experiment_name = exp_name,
#                                               lr_scheduler_ctor=lambda optim: torch.optim.lr_scheduler.ReduceLROnPlateau(optim, patience=hyps['lr_sched_pat'],
#                                                                                                                          factor=hyps['lr_sched_fac'],
#                                                                                                                          verbose=True))

In [ ]:
# class_weights_manual = torch.tensor(np.ones(len(np.unique(train_labels.data))), dtype=torch.float).cuda()
# class_weights_manual[0] = 0.0001
# class_weights_manual

In [ ]:
# #hyperparameters
# exp_name = 'class_weights_manual_01'
# hyps = {
#     'dataset': 'base',
#     'char_min_count': 5,
#     'char_max_doc_freq': 1.0, 
#     'MAX_SENT_LEN': MAX_SENT_LEN,
#     'MAX_ORIG_TOKEN_LEN': MAX_ORIG_TOKEN_LEN,
#     'start_end_tag': False,
    
#     #TokenPOSTagger
#     'emb_size': 64,
#     'res_block': wisResidualBlock,
#     'sng_backbone_kwargs': {
#         'layers_n': 3, 
#         'kernel_size': 3, 
#         'dropout': 0.3,
#         'conv_layer': MyConv1d,
#         'batchnorm': None,
#         'dilation': False },
#     'ctx_backbone_kwargs': {
#         'layers_n': 3,
#         'kernel_size': 3,
#         'dropout': 0.3,
#         'conv_layer': MyConv1d,
#         'batchnorm': None,
#         'dilation': False  },

#     #train_eval_loop
#     'loss_func': nn.CrossEntropyLoss(weight=class_weights_manual,reduction='mean'),
#     'lr': 5e-3,
#     'epoch_n': 30,
#     'batch': 64,
#     'device': 'cuda',
#     'stop_pat': 5,
#     'lr_sched_pat': 2,
#     'lr_sched_fac': 0.5
# }

# histories[exp_name] = {
#     'hyps': hyps
# }

# train_char_tokenized = tokenize_corpus(all_train_texts, tokenizer=character_tokenize)
# char_vocab, word_doc_freq = build_vocabulary(train_char_tokenized, 
#                                              max_doc_freq=hyps['char_max_doc_freq'], 
#                                              min_count=hyps['char_min_count'], 
#                                              pad_word='<PAD>', 
#                                              start_end_tag=hyps['start_end_tag'])

# print("Количество уникальных символов", len(char_vocab))
# print(list(char_vocab.items())[:10])


# UNIQUE_TAGS = ['<NOTAG>'] + sorted({token.upos for sent in full_train for token in sent if token.upos})
# label2id = {label: i for i, label in enumerate(UNIQUE_TAGS)}

# train_inputs, train_labels = pos_corpus_to_tensor(full_train, char_vocab, label2id, MAX_SENT_LEN, MAX_ORIG_TOKEN_LEN)
# train_dataset = TensorDataset(train_inputs, train_labels)

# test_inputs, test_labels = pos_corpus_to_tensor(full_test, char_vocab, label2id, MAX_SENT_LEN, MAX_ORIG_TOKEN_LEN)
# test_dataset = TensorDataset(test_inputs, test_labels)

# if hyps['start_end_tag']:
#     #for calc final statistics
#     hdatasets['setags'] = {}
#     hdatasets['setags']['char_vocab'] = char_vocab
#     hdatasets['setags']['train_labels'] = train_labels
#     hdatasets['setags']['train_dataset'] = train_dataset
#     hdatasets['setags']['test_labels'] = test_labels
#     hdatasets['setags']['test_dataset'] = test_dataset

# sentence_level_model = SentenceLevelPOSTagger(len(char_vocab), len(label2id), 
#                                               embedding_size=hyps['emb_size'], res_block=hyps['res_block'],
#                                               single_backbone_kwargs=hyps['sng_backbone_kwargs'],
#                                               context_backbone_kwargs=hyps['ctx_backbone_kwargs'])
# print('Количество параметров', sum(np.product(t.shape) for t in sentence_level_model.parameters()))

# (histories[exp_name]['train_history'],
#  histories[exp_name]['best_model']) = train_eval_loop(sentence_level_model,
#                                               train_dataset,
#                                               test_dataset,
#                                               hyps['loss_func'],
#                                               lr=hyps['lr'],
#                                               epoch_n=hyps['epoch_n'],
#                                               batch_size=hyps['batch'],
#                                               device=hyps['device'],
#                                               early_stopping_patience=hyps['stop_pat'],
#                                               max_batches_per_epoch_train=500,
#                                               max_batches_per_epoch_val=300,
#                                               best_acc_type = 'acc',
#                                               experiment_name = exp_name,
#                                               lr_scheduler_ctor=lambda optim: torch.optim.lr_scheduler.ReduceLROnPlateau(optim, patience=hyps['lr_sched_pat'],
#                                                                                                                          factor=hyps['lr_sched_fac'],
#                                                                                                                          verbose=True))

### Итоговые эксперименты

In [ ]:
# #hyperparameters
# exp_name = 'lr=1e-2_emb_size=100_start_end_tags_sng_layers_n=12'
# hyps = {
#     'dataset': 'setags',
#     'char_min_count': 5,
#     'char_max_doc_freq': 1.0, 
#     'MAX_SENT_LEN': MAX_SENT_LEN,
#     'MAX_ORIG_TOKEN_LEN': MAX_ORIG_TOKEN_LEN,
#     'start_end_tag': True,
    
#     #TokenPOSTagger
#     'emb_size': 100,
#     'res_block': wisResidualBlock,
#     'sng_backbone_kwargs': {
#         'layers_n': 12, 
#         'kernel_size': 3, 
#         'dropout': 0.3,
#         'conv_layer': MyConv1d,
#         'batchnorm': None,
#         'dilation': False },
#     'ctx_backbone_kwargs': {
#         'layers_n': 3,
#         'kernel_size': 3,
#         'dropout': 0.3,
#         'conv_layer': MyConv1d,
#         'batchnorm': None,
#         'dilation': False  },

#     #train_eval_loop
#     'loss_func': F.cross_entropy,
#     'lr': 1e-2,
#     'epoch_n': 70,
#     'batch': 64,
#     'device': 'cuda',
#     'stop_pat': 5,
#     'lr_sched_pat': 2,
#     'lr_sched_fac': 0.5
# }

# histories[exp_name] = {
#     'hyps': hyps
# }

# train_char_tokenized = tokenize_corpus(all_train_texts, tokenizer=character_tokenize)
# char_vocab, word_doc_freq = build_vocabulary(train_char_tokenized, 
#                                              max_doc_freq=hyps['char_max_doc_freq'], 
#                                              min_count=hyps['char_min_count'], 
#                                              pad_word='<PAD>', 
#                                              start_end_tag=hyps['start_end_tag'])

# print("Количество уникальных символов", len(char_vocab))
# print(list(char_vocab.items())[:10])


# UNIQUE_TAGS = ['<NOTAG>'] + sorted({token.upos for sent in full_train for token in sent if token.upos})
# label2id = {label: i for i, label in enumerate(UNIQUE_TAGS)}

# train_inputs, train_labels = pos_corpus_to_tensor(full_train, char_vocab, label2id, MAX_SENT_LEN, MAX_ORIG_TOKEN_LEN)
# train_dataset = TensorDataset(train_inputs, train_labels)

# test_inputs, test_labels = pos_corpus_to_tensor(full_test, char_vocab, label2id, MAX_SENT_LEN, MAX_ORIG_TOKEN_LEN)
# test_dataset = TensorDataset(test_inputs, test_labels)

# if hyps['start_end_tag']:
#     #for calc final statistics
#     hdatasets['setags'] = {}
#     hdatasets['setags']['char_vocab'] = char_vocab
#     hdatasets['setags']['train_labels'] = train_labels
#     hdatasets['setags']['train_dataset'] = train_dataset
#     hdatasets['setags']['test_labels'] = test_labels
#     hdatasets['setags']['test_dataset'] = test_dataset

# sentence_level_model = SentenceLevelPOSTagger(len(char_vocab), len(label2id), 
#                                               embedding_size=hyps['emb_size'], res_block=hyps['res_block'],
#                                               single_backbone_kwargs=hyps['sng_backbone_kwargs'],
#                                               context_backbone_kwargs=hyps['ctx_backbone_kwargs'])
# print('Количество параметров', sum(np.product(t.shape) for t in sentence_level_model.parameters()))

# (histories[exp_name]['train_history'],
#  histories[exp_name]['best_model']) = train_eval_loop(sentence_level_model,
#                                               train_dataset,
#                                               test_dataset,
#                                               hyps['loss_func'],
#                                               lr=hyps['lr'],
#                                               epoch_n=hyps['epoch_n'],
#                                               batch_size=hyps['batch'],
#                                               device=hyps['device'],
#                                               early_stopping_patience=hyps['stop_pat'],
#                                               max_batches_per_epoch_train=500,
#                                               max_batches_per_epoch_val=300,
#                                               best_acc_type = 'acc',
#                                               experiment_name = exp_name,
#                                               lr_scheduler_ctor=lambda optim: torch.optim.lr_scheduler.ReduceLROnPlateau(optim, patience=hyps['lr_sched_pat'],
#                                                                                                                          factor=hyps['lr_sched_fac'],
#                                                                                                                          verbose=True))

In [ ]:
# #hyperparameters
# exp_name = 'lr=1e-2_emb_size=100_start_end_tags_all_layers_n=12'
# hyps = {
#     'dataset': 'setags',
#     'char_min_count': 5,
#     'char_max_doc_freq': 1.0, 
#     'MAX_SENT_LEN': MAX_SENT_LEN,
#     'MAX_ORIG_TOKEN_LEN': MAX_ORIG_TOKEN_LEN,
#     'start_end_tag': True,
    
#     #TokenPOSTagger
#     'emb_size': 100,
#     'res_block': wisResidualBlock,
#     'sng_backbone_kwargs': {
#         'layers_n': 12, 
#         'kernel_size': 3, 
#         'dropout': 0.3,
#         'conv_layer': MyConv1d,
#         'batchnorm': None,
#         'dilation': False },
#     'ctx_backbone_kwargs': {
#         'layers_n': 12,
#         'kernel_size': 3,
#         'dropout': 0.3,
#         'conv_layer': MyConv1d,
#         'batchnorm': None,
#         'dilation': False  },

#     #train_eval_loop
#     'loss_func': F.cross_entropy,
#     'lr': 1e-2,
#     'epoch_n': 60,
#     'batch': 64,
#     'device': 'cuda',
#     'stop_pat': 5,
#     'lr_sched_pat': 2,
#     'lr_sched_fac': 0.5
# }

# histories[exp_name] = {
#     'hyps': hyps
# }

# train_char_tokenized = tokenize_corpus(all_train_texts, tokenizer=character_tokenize)
# char_vocab, word_doc_freq = build_vocabulary(train_char_tokenized, 
#                                              max_doc_freq=hyps['char_max_doc_freq'], 
#                                              min_count=hyps['char_min_count'], 
#                                              pad_word='<PAD>', 
#                                              start_end_tag=hyps['start_end_tag'])

# print("Количество уникальных символов", len(char_vocab))
# print(list(char_vocab.items())[:10])


# UNIQUE_TAGS = ['<NOTAG>'] + sorted({token.upos for sent in full_train for token in sent if token.upos})
# label2id = {label: i for i, label in enumerate(UNIQUE_TAGS)}

# train_inputs, train_labels = pos_corpus_to_tensor(full_train, char_vocab, label2id, MAX_SENT_LEN, MAX_ORIG_TOKEN_LEN)
# train_dataset = TensorDataset(train_inputs, train_labels)

# test_inputs, test_labels = pos_corpus_to_tensor(full_test, char_vocab, label2id, MAX_SENT_LEN, MAX_ORIG_TOKEN_LEN)
# test_dataset = TensorDataset(test_inputs, test_labels)

# if hyps['start_end_tag']:
#     #for calc final statistics
#     hdatasets['setags'] = {}
#     hdatasets['setags']['char_vocab'] = char_vocab
#     hdatasets['setags']['train_labels'] = train_labels
#     hdatasets['setags']['train_dataset'] = train_dataset
#     hdatasets['setags']['test_labels'] = test_labels
#     hdatasets['setags']['test_dataset'] = test_dataset

# sentence_level_model = SentenceLevelPOSTagger(len(char_vocab), len(label2id), 
#                                               embedding_size=hyps['emb_size'], res_block=hyps['res_block'],
#                                               single_backbone_kwargs=hyps['sng_backbone_kwargs'],
#                                               context_backbone_kwargs=hyps['ctx_backbone_kwargs'])
# print('Количество параметров', sum(np.product(t.shape) for t in sentence_level_model.parameters()))

# (histories[exp_name]['train_history'],
#  histories[exp_name]['best_model']) = train_eval_loop(sentence_level_model,
#                                               train_dataset,
#                                               test_dataset,
#                                               hyps['loss_func'],
#                                               lr=hyps['lr'],
#                                               epoch_n=hyps['epoch_n'],
#                                               batch_size=hyps['batch'],
#                                               device=hyps['device'],
#                                               early_stopping_patience=hyps['stop_pat'],
#                                               max_batches_per_epoch_train=500,
#                                               max_batches_per_epoch_val=300,
#                                               best_acc_type = 'acc',
#                                               experiment_name = exp_name,
#                                               lr_scheduler_ctor=lambda optim: torch.optim.lr_scheduler.ReduceLROnPlateau(optim, patience=hyps['lr_sched_pat'],
#                                                                                                                          factor=hyps['lr_sched_fac'],
#                                                                                                                          verbose=True))

In [ ]:
# #hyperparameters
# exp_name = 'lr=1e-2_emb_size=100_start_end_tags_sng_layers_n=12_kernel_size=5'
# hyps = {
#     'dataset': 'setags',
#     'char_min_count': 5,
#     'char_max_doc_freq': 1.0, 
#     'MAX_SENT_LEN': MAX_SENT_LEN,
#     'MAX_ORIG_TOKEN_LEN': MAX_ORIG_TOKEN_LEN,
#     'start_end_tag': True,
    
#     #TokenPOSTagger
#     'emb_size': 100,
#     'res_block': wisResidualBlock,
#     'sng_backbone_kwargs': {
#         'layers_n': 12, 
#         'kernel_size': 5, 
#         'dropout': 0.3,
#         'conv_layer': MyConv1d,
#         'batchnorm': None,
#         'dilation': False },
#     'ctx_backbone_kwargs': {
#         'layers_n': 3,
#         'kernel_size': 5,
#         'dropout': 0.3,
#         'conv_layer': MyConv1d,
#         'batchnorm': None,
#         'dilation': False  },

#     #train_eval_loop
#     'loss_func': F.cross_entropy,
#     'lr': 1e-2,
#     'epoch_n': 40,
#     'batch': 64,
#     'device': 'cuda',
#     'stop_pat': 5,
#     'lr_sched_pat': 2,
#     'lr_sched_fac': 0.5
# }

# histories[exp_name] = {
#     'hyps': hyps
# }

# train_char_tokenized = tokenize_corpus(all_train_texts, tokenizer=character_tokenize)
# char_vocab, word_doc_freq = build_vocabulary(train_char_tokenized, 
#                                              max_doc_freq=hyps['char_max_doc_freq'], 
#                                              min_count=hyps['char_min_count'], 
#                                              pad_word='<PAD>', 
#                                              start_end_tag=hyps['start_end_tag'])

# print("Количество уникальных символов", len(char_vocab))
# print(list(char_vocab.items())[:10])


# UNIQUE_TAGS = ['<NOTAG>'] + sorted({token.upos for sent in full_train for token in sent if token.upos})
# label2id = {label: i for i, label in enumerate(UNIQUE_TAGS)}

# train_inputs, train_labels = pos_corpus_to_tensor(full_train, char_vocab, label2id, MAX_SENT_LEN, MAX_ORIG_TOKEN_LEN)
# train_dataset = TensorDataset(train_inputs, train_labels)

# test_inputs, test_labels = pos_corpus_to_tensor(full_test, char_vocab, label2id, MAX_SENT_LEN, MAX_ORIG_TOKEN_LEN)
# test_dataset = TensorDataset(test_inputs, test_labels)

# if hyps['start_end_tag']:
#     #for calc final statistics
#     hdatasets['setags'] = {}
#     hdatasets['setags']['char_vocab'] = char_vocab
#     hdatasets['setags']['train_labels'] = train_labels
#     hdatasets['setags']['train_dataset'] = train_dataset
#     hdatasets['setags']['test_labels'] = test_labels
#     hdatasets['setags']['test_dataset'] = test_dataset

# sentence_level_model = SentenceLevelPOSTagger(len(char_vocab), len(label2id), 
#                                               embedding_size=hyps['emb_size'], res_block=hyps['res_block'],
#                                               single_backbone_kwargs=hyps['sng_backbone_kwargs'],
#                                               context_backbone_kwargs=hyps['ctx_backbone_kwargs'])
# print('Количество параметров', sum(np.product(t.shape) for t in sentence_level_model.parameters()))

# (histories[exp_name]['train_history'],
#  histories[exp_name]['best_model']) = train_eval_loop(sentence_level_model,
#                                               train_dataset,
#                                               test_dataset,
#                                               hyps['loss_func'],
#                                               lr=hyps['lr'],
#                                               epoch_n=hyps['epoch_n'],
#                                               batch_size=hyps['batch'],
#                                               device=hyps['device'],
#                                               early_stopping_patience=hyps['stop_pat'],
#                                               max_batches_per_epoch_train=500,
#                                               max_batches_per_epoch_val=300,
#                                               best_acc_type = 'acc',
#                                               experiment_name = exp_name,
#                                               lr_scheduler_ctor=lambda optim: torch.optim.lr_scheduler.ReduceLROnPlateau(optim, patience=hyps['lr_sched_pat'],
#                                                                                                                          factor=hyps['lr_sched_fac'],
#                                                                                                                          verbose=True))

### Еще один эксперимент, иная инициализация весов эмбеддингов

In [ ]:
class SentenceLevelPOSTagger(nn.Module):
    #v1.1 Custom init embeddings
    def __init__(self, vocab_size, labels_num, embedding_size=32, res_block = wisResidualBlock, 
                 single_backbone_kwargs={}, context_backbone_kwargs={}):
        super().__init__()
        self.embedding_size = embedding_size
        self.char_embeddings = nn.Embedding(vocab_size, embedding_size, padding_idx=0)
        self.char_embeddings.weight.data.uniform_(-1.0 / embedding_size, 1.0 / embedding_size)
        self.char_embeddings.weight.data[0] = 0
        
        self.single_token_backbone = wisStackedConv1d(embedding_size, res_block=res_block, **single_backbone_kwargs)
        self.context_backbone = wisStackedConv1d(embedding_size, res_block=res_block, **context_backbone_kwargs)
        self.global_pooling = nn.AdaptiveMaxPool1d(1)
        self.out = nn.Conv1d(embedding_size, labels_num, 1)
        self.labels_num = labels_num
    
    def forward(self, tokens):
        """tokens - BatchSize x MaxSentenceLen x MaxTokenLen"""
        batch_size, max_sent_len, max_token_len = tokens.shape
        tokens_flat = tokens.view(batch_size * max_sent_len, max_token_len)
        
        char_embeddings = self.char_embeddings(tokens_flat)  # BatchSize*MaxSentenceLen x MaxTokenLen x EmbSize
        char_embeddings = char_embeddings.permute(0, 2, 1)  # BatchSize*MaxSentenceLen x EmbSize x MaxTokenLen
        char_features = self.single_token_backbone(char_embeddings)
        
        token_features_flat = self.global_pooling(char_features).squeeze(-1)  # BatchSize*MaxSentenceLen x EmbSize

        token_features = token_features_flat.view(batch_size, max_sent_len, self.embedding_size)  # BatchSize x MaxSentenceLen x EmbSize
        token_features = token_features.permute(0, 2, 1)  # BatchSize x EmbSize x MaxSentenceLen
        context_features = self.context_backbone(token_features)  # BatchSize x EmbSize x MaxSentenceLen

        logits = self.out(context_features)  # BatchSize x LabelsNum x MaxSentenceLen
        return logits

In [ ]:
#hyperparameters
exp_name = 'custom_init_embeddings_weights'
hyps = {
    'dataset': 'base',
    'char_min_count': 5,
    'char_max_doc_freq': 1.0, 
    'MAX_SENT_LEN': MAX_SENT_LEN,
    'MAX_ORIG_TOKEN_LEN': MAX_ORIG_TOKEN_LEN,
    
    #TokenPOSTagger
    'emb_size': 64,
    'res_block': wisResidualBlock,
    'sng_backbone_kwargs': {
        'layers_n': 3, 
        'kernel_size': 3, 
        'dropout': 0.3,
        'conv_layer': MyConv1d,
        'batchnorm': None },
    'ctx_backbone_kwargs': {
        'layers_n': 3, 
        'kernel_size': 3, 
        'dropout': 0.3,
        'conv_layer': MyConv1d,
        'batchnorm': None },

    #train_eval_loop
    'lr': 5e-3,
    'epoch_n': 30,
    'batch': 64,
    'device': 'cuda',
    'stop_pat': 5,
    'lr_sched_pat': 2,
    'lr_sched_fac': 0.5
}

histories[exp_name] = {
    'hyps': hyps
}


sentence_level_model = SentenceLevelPOSTagger(len(char_vocab), len(label2id), 
                                              embedding_size=hyps['emb_size'], res_block=hyps['res_block'],
                                              single_backbone_kwargs=hyps['sng_backbone_kwargs'],
                                              context_backbone_kwargs=hyps['ctx_backbone_kwargs'])
print('Количество параметров', sum(np.product(t.shape) for t in sentence_level_model.parameters()))

(histories[exp_name]['train_history'],
 histories[exp_name]['best_model']) = train_eval_loop(sentence_level_model,
                                              train_dataset,
                                              test_dataset,
                                              F.cross_entropy,
                                              lr=hyps['lr'],
                                              epoch_n=hyps['epoch_n'],
                                              batch_size=hyps['batch'],
                                              device=hyps['device'],
                                              early_stopping_patience=hyps['stop_pat'],
                                              max_batches_per_epoch_train=500,
                                              max_batches_per_epoch_val=300,
                                              best_acc_type = 'acc',
                                              experiment_name = exp_name,
                                              lr_scheduler_ctor=lambda optim: torch.optim.lr_scheduler.ReduceLROnPlateau(optim, patience=hyps['lr_sched_pat'],
                                                                                                                         factor=hyps['lr_sched_fac'],
                                                                                                                         verbose=True))

## Итоги экспериментов

In [ ]:
#show_general_experiments_stats
train_history = {}
for exp_name, exp in histories.items():
    if 'train_history' in exp:
        train_history[exp_name] = exp['train_history']
show_experiments_stats(train_history, show_plots = True, only_BEST_MODEL_CALC = False)

In [ ]:
#show_reports_stats
for exp_name in histories:
    print('{:-<100}'.format(exp_name))
    #choise saved dataset type
    dtype = histories[exp_name]['hyps']['dataset']
    train_labels = hdatasets[dtype]['train_labels']
    train_dataset = hdatasets[dtype]['train_dataset']
    test_labels = hdatasets[dtype]['test_labels']
    test_dataset = hdatasets[dtype]['test_dataset']
    
    train_pred = predict_with_model(histories[exp_name]['best_model'], train_dataset)
    train_loss = F.cross_entropy(torch.tensor(train_pred),
                                              torch.tensor(train_labels))
    print('Среднее значение функции потерь на обучении', float(train_loss))
    print(classification_report(train_labels.view(-1), train_pred.argmax(1).reshape(-1), target_names=UNIQUE_TAGS))
    histories[exp_name]['f1_macro'] = {}
    histories[exp_name]['f1_macro']['train'] = f1_score(train_labels.view(-1), train_pred.argmax(1).reshape(-1), average='macro')
    print()

    test_pred = predict_with_model(histories[exp_name]['best_model'], test_dataset)
    test_loss = F.cross_entropy(torch.tensor(test_pred),
                                torch.tensor(test_labels))
    print('Среднее значение функции потерь на валидации', float(test_loss))
    print(classification_report(test_labels.view(-1), test_pred.argmax(1).reshape(-1), target_names=UNIQUE_TAGS))
    histories[exp_name]['f1_macro']['val'] = f1_score(test_labels.view(-1), test_pred.argmax(1).reshape(-1), average='macro')
    print()

print()
for exp_name in histories:
    print('{:-<100}'.format(exp_name))
    print('f1_macro: Train = {:.3f}, Val = {:.3f}'.format(histories[exp_name]['f1_macro']['train'],
                                                          histories[exp_name]['f1_macro']['val']))

### Применение полученных теггеров и сравнение на тестовых предложениях

In [ ]:
import re
test_sentences = [
    'Мама мыла раму.',
    'Косил косой косой косой.',
    'Глокая куздра штеко будланула бокра и куздрячит бокрёнка.',
    'Сяпала Калуша с Калушатами по напушке.',
    'Пирожки поставлены в печь, мама любит печь.',
    'Ведро дало течь, вода стала течь.',
    'Три да три, будет дырка.',
    'Три да три, будет шесть.',
    'Сорок сорок.'
]
test_sentences_tokenized = tokenize_corpus(test_sentences, min_token_size=1)

test_sentences_with_true_pos = [
    'мама-NOUN мыла-VERB раму-NOUN',
    'косил-VERB косой-NOUN косой-ADJ косой-NOUN',
    'глокая-ADJ куздра-NOUN штеко-ADV будланула-VERB бокра-NOUN и-CCONJ куздрячит-VERB бокрёнка-NOUN',
    'сяпала-VERB калуша-NOUN с-ADP калушатами-NOUN по-ADP напушке-NOUN',
    'пирожки-NOUN поставлены-VERB в-ADP печь-NOUN мама-NOUN любит-VERB печь-VERB',
    'ведро-NOUN дало-VERB течь-NOUN вода-NOUN стала-VERB течь-VERB',
    'три-NUM да-CCONJ три-NUM будет-AUX дырка-NOUN',
    'три-NUM да-CCONJ три-NUM будет-AUX шесть-NUM',
    'сорок-NUM сорок-NOUN'
]
pos_pattern = r"[A-Za-z]+"
test_sentences_true_pos = [re.findall(pos_pattern, sent_with_pos) for sent_with_pos in test_sentences_with_true_pos]

for exp_name in histories:
    print('{:-<100}'.format(exp_name))
    print('')
    
    #choise saved dataset/vocab type
    dtype = histories[exp_name]['hyps']['dataset']
    char_vocab = hdatasets[dtype]['char_vocab']
    
    token_pos_tagger = POSTagger(histories[exp_name]['best_model'], char_vocab, UNIQUE_TAGS, 
                                 histories[exp_name]['hyps']['MAX_SENT_LEN'], histories[exp_name]['hyps']['MAX_ORIG_TOKEN_LEN'])
    correct_predicts = []
    for sent_tokens, sent_tags, sent_true_tags in zip(test_sentences_tokenized, token_pos_tagger(test_sentences), test_sentences_true_pos):
        out_tags = []
        for tag, true_tag in zip(sent_tags, sent_true_tags):
            correct_predicts.append(tag == true_tag)
            if correct_predicts[-1]:
                out_tags.append(tag)
            else:
                out_tags.append("{}!BAD!->{}".format(tag, true_tag))
                         
        print(' '.join('{}-{}'.format(tok, tag) for tok, tag in zip(sent_tokens, out_tags)))
        print()
    histories[exp_name]['TEST_SENTENS'] = {
        'ACC': np.mean(np.array(correct_predicts)),
        'correct': sum(np.array(correct_predicts)),
        'all': len(correct_predicts)
    }
    print('ACC: {:.4f}, correct = {} of {}'.format(histories[exp_name]['TEST_SENTENS']['ACC'], 
                                                  histories[exp_name]['TEST_SENTENS']['correct'], 
                                                  histories[exp_name]['TEST_SENTENS']['all']))
    print('')
    print('')

for exp_name in histories:
    print('{:-<100}'.format(exp_name))
    print('ACC: {:.4f}, correct = {} of {}'.format(histories[exp_name]['TEST_SENTENS']['ACC'], 
                                                  histories[exp_name]['TEST_SENTENS']['correct'], 
                                                  histories[exp_name]['TEST_SENTENS']['all']))